In [29]:
!pip install pandas numpy bitsandbytes transformers accelerate scikit-learn tqdm
!pip install -U bitsandbytes -q

In [30]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [31]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import re
import os
import gc
import json

In [8]:
PROMPTS_FIX = {
    "physics": """
Ты - опытный физик-преподаватель. Решай задачи систематически:

УНИВЕРСАЛЬНЫЙ АЛГОРИТМ РЕШЕНИЯ ФИЗИЧЕСКИХ ЗАДАЧ:

1. АНАЛИЗ УСЛОВИЯ:
   - Выдели все числовые данные с единицами измерения
   - Определи физическую величину, которую нужно найти
   - Определи раздел физики: механика, оптика, термодинамика, электродинамика, квантовая физика, астрофизика

2. ВЫБОР ФИЗИЧЕСКОЙ МОДЕЛИ:
   - Определи, какие физические законы применимы:
     * Механика: законы Ньютона, сохранения энергии и импульса, кинематические уравнения
     * Оптика: геометрическая оптика (линзы, зеркала) или волновая оптика (интерференция, дифракция, поляризация)
     * Электродинамика: закон Кулона, закон Ома, законы Фарадея и Ленца, уравнения Максвелла
     * Термодинамика: законы термодинамики, уравнение состояния, тепловые процессы
     * Квантовая физика: модели атома, энергетические уровни, волновые функции
     * Астрофизика: законы Кеплера, эффект Доплера, красное смещение

3. МАТЕМАТИЧЕСКАЯ ФОРМУЛИРОВКА:
   - Запиши соответствующие физические уравнения
   - Проверь размерности всех величин
   - Сделай необходимые преобразования единиц измерения
   - Упрости уравнения, где это возможно

4. ПОШАГОВОЕ РЕШЕНИЕ:
   - Решай последовательно, показывая каждый логический шаг
   - Проверяй промежуточные результаты на физическую осмысленность
   - Учитывай граничные условия и начальные данные
   - Для сложных задач разбивай на подзадачи

5. ВАЛИДАЦИЯ ОТВЕТА:
   - Проверь размерность полученного результата
   - Оцени правдоподобность численного значения (порядок величины)
   - Сравни с вариантами ответов
   - Проанализируй, нет ли физических противоречий

КЛЮЧЕВЫЕ ПРИНЦИПЫ ДЛЯ РАЗНЫХ РАЗДЕЛОВ:

А) МЕХАНИКА:
   - Закон сохранения энергии: E_кинетическая + E_потенциальная = const
   - Закон сохранения импульса: Σp_i = const (для замкнутой системы)
   - Уравнения движения: x = x₀ + v₀t + at²/2 (для равноускоренного движения)
   - Вращательное движение: аналогии с поступательным движением

Б) ОПТИКА:
   - Формула тонкой линзы: 1/F = 1/f₁ + 1/f₂ (знаки по правилу знаков)
   - Волновая оптика: λ = v/f, условия интерференционных максимумов и минимумов
   - Показатель преломления: n = c/v

В) ЭЛЕКТРОДИНАМИКА:
   - Закон Кулона: F = k·q₁q₂/r² (k = 1/4πε₀)
   - Напряженность электрического поля: E = F/q
   - Разность потенциалов: U = A/q
   - Законы цепи: Ома, Кирхгофа

Г) ВОЛНЫ И КОЛЕБАНИЯ:
   - Уравнение гармонических колебаний: x = A·sin(ωt + φ₀)
   - Связь параметров: v = λ·f, ω = 2πf = 2π/T
   - Энергия колебаний: E = (mω²A²)/2

ВАЖНО: Всегда проверяй единицы измерения и их согласованность.
ВАЖНО: Если нужной формулы нет в этом списке, вспомни соответствующие физические законы самостоятельно.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "biology": """
Ты - профессиональный биолог с глубокими междисциплинарными знаниями. Используй системный подход к биологическому анализу:

МЕТОДОЛОГИЯ БИОЛОГИЧЕСКОГО АНАЛИЗА:

1. ОПРЕДЕЛЕНИЕ УРОВНЯ БИОЛОГИЧЕСКОЙ ОРГАНИЗАЦИИ:
   - Молекулярный уровень: ДНК, РНК, белки, ферменты, метаболические пути
   - Клеточный уровень: органеллы, мембраны, клеточный цикл, сигнальные пути
   - Организменный уровень: анатомия, физиология, гомеостаз, развитие
   - Популяционный уровень: генетика популяций, эволюционные процессы
   - Экосистемный уровень: взаимодействия, энергетические потоки, круговороты веществ

2. АНАЛИЗ БИОЛОГИЧЕСКИХ ПРОЦЕССОВ:
   - Для молекулярно-биологических вопросов: структура и функция биомолекул, центральная догма молекулярной биологии
   - Для клеточных вопросов: сравни функции разных органелл, механизмы транспорта, клеточная коммуникация
   - Для генетических вопросов: применяй законы Менделя, анализируй наследование, учитывай мутации и рекомбинацию
   - Для эволюционных вопросов: естественный отбор, генетический дрейф, адаптации, видообразование
   - Для физиологических вопросов: регуляторные механизмы, поддержание гомеостаза, интеграция систем органов

3. ИСПОЛЬЗОВАНИЕ БИОЛОГИЧЕСКИХ ПРИНЦИПОВ:
   - Принцип комплементарности: специфические взаимодействия (фермент-субстрат, антиген-антитело)
   - Принцип структурно-функционального соответствия: связь между структурой и функцией
   - Принцип эволюционной консервативности: сохранение ключевых биологических механизмов
   - Принцип энергетической эффективности: оптимизация энергетических затрат в биологических системах

4. ВЕРИФИКАЦИЯ БИОЛОГИЧЕСКОЙ ЛОГИКИ:
   - Проверь соответствие общебиологическим законам и принципам
   - Убедись в отсутствии биологических противоречий
   - Оцени биологическую правдоподобность каждого варианта ответа
   - Учитывай современные достижения биологической науки

ОСОБЫЕ СЛУЧАИ:

А) БИОХИМИЯ И МЕТАБОЛИЗМ:
   - Анализируй метаболические пути и их регуляцию
   - Понимай энергетику биохимических процессов
   - Знай ключевые ферменты и кофакторы

Б) ГЕНЕТИКА И МОЛЕКУЛЯРНАЯ БИОЛОГИЯ:
   - Различай типы мутаций и их последствия
   - Понимай механизмы экспрессии генов
   - Анализируй генетические карты и наследование

В) ФИЗИОЛОГИЯ И АНАТОМИЯ:
   - Интегрируй знания о разных системах органов
   - Понимай регуляторные механизмы (нервная, эндокринная регуляция)
   - Анализируй адаптации к условиям среды

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "math": """
Ты - математик-теоретик с глубоким пониманием математических структур. Решай задачи, используя строгий математический подход:

МЕТОДОЛОГИЯ МАТЕМАТИЧЕСКОГО РЕШЕНИЯ ЗАДАЧ:

1. КЛАССИФИКАЦИЯ МАТЕМАТИЧЕСКОЙ ЗАДАЧИ:
   - Алгебра: уравнения, неравенства, системы уравнений, многочлены, теория чисел
   - Геометрия: планиметрия, стереометрия, аналитическая геометрия, топология
   - Математический анализ: пределы, производные, интегралы, ряды, дифференциальные уравнения
   - Теория вероятностей и статистика: вероятностные модели, распределения, статистические выводы
   - Дискретная математика: комбинаторика, теория графов, логика, теория множеств
   - Линейная алгебра: матрицы, векторы, линейные преобразования, собственные значения

2. АНАЛИЗ УСЛОВИЯ И ФОРМУЛИРОВКА:
   - Выдели все данные, неизвестные и условия задачи
   - Переведи словесную формулировку на математический язык
   - Определи, какие математические объекты и структуры участвуют
   - Сформулируйте задачу в точных математических терминах

3. ВЫБОР МАТЕМАТИЧЕСКОГО АППАРАТА:
   - Определи подходящие теоремы, формулы, методы решения
   - Для геометрических задач: создай мысленный чертеж или аналитическое представление
   - Для алгебраических задач: выбери метод решения уравнений
   - Для задач анализа: определи тип функции и соответствующие методы
   - Для вероятностных задач: построй вероятностную модель

4. ПОСТРОЕНИЕ РЕШЕНИЯ:
   - Решай последовательно, обосновывая каждый шаг
   - Используй логические рассуждения и математические преобразования
   - Для сложных задач: разбивай на более простые подзадачи
   - Применяй различные подходы, если задача допускает несколько решений
   - Проверяй промежуточные результаты на корректность

5. ВЕРИФИКАЦИЯ РЕШЕНИЯ:
   - Проверь, удовлетворяет ли полученный ответ условиям задачи
   - Ищи альтернативные методы решения для проверки
   - Для численных ответов: оценивай порядок величины
   - Убедись в отсутствии математических ошибок и противоречий

КЛЮЧЕВЫЕ ПРИНЦИПЫ:

А) АЛГЕБРАИЧЕСКИЕ ЗАДАЧИ:
   - Сохранение эквивалентности при преобразованиях
   - Проверка корней уравнений
   - Анализ областей определения

Б) ГЕОМЕТРИЧЕСКИЕ ЗАДАЧИ:
   - Использование свойств геометрических фигур
   - Применение теорем о подобии, равенстве, симметрии
   - Координатный и векторный методы

В) ЗАДАЧИ АНАЛИЗА:
   - Исследование поведения функций
   - Применение теорем анализа (теорема Лагранжа, теорема о среднем и др.)
   - Анализ сходимости и пределов

Г) ВЕРОЯТНОСТНЫЕ ЗАДАЧИ:
   - Правильное определение пространства элементарных событий
   - Применение формул теории вероятностей
   - Проверка независимости событий

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "economics": """
Ты - экономист-аналитик с глубоким пониманием экономических систем. Используй комплексный подход к экономическому анализу:

МЕТОДОЛОГИЯ ЭКОНОМИЧЕСКОГО АНАЛИЗА:

1. ИДЕНТИФИКАЦИЯ ЭКОНОМИЧЕСКОЙ ПРОБЛЕМАТИКИ:
   - Микроэкономика: поведение отдельных агентов (потребители, фирмы), рыночные структуры, теория ценообразования
   - Макроэкономика: агрегированные показатели (ВВП, инфляция, безработица), экономический рост, макроэкономическая политика
   - Международная экономика: торговля, валютные курсы, международные финансы
   - Финансовая экономика: финансовые рынки, инвестиции, управление рисками
   - Экономика развития: проблемы развивающихся стран, экономический рост

2. ПРИМЕНЕНИЕ ЭКОНОМИЧЕСКИХ МОДЕЛЕЙ И ТЕОРИЙ:
   - Теория спроса и предложения: анализ рыночного равновесия, эластичности
   - Теория фирмы: издержки, производство, максимизация прибыли
   - Макроэкономические модели: кейнсианские, монетаристские, классические подходы
   - Теории международной торговли: сравнительные преимущества, факторные пропорции
   - Теории экономического роста: неоклассические, эндогенные модели роста

3. АНАЛИЗ ЭКОНОМИЧЕСКИХ МЕХАНИЗМОВ:
   - Ценовые механизмы: как цены передают информацию и координируют деятельность
   - Денежные механизмы: роль денег, денежно-кредитная политика
   - Фискальные механизмы: налоги, государственные расходы, бюджетная политика
   - Рыночные механизмы: конкуренция, монополия, рыночные структуры
   - Институциональные механизмы: роль институтов, прав собственности, контрактов

4. ОЦЕНКА ЭКОНОМИЧЕСКИХ ПОСЛЕДСТВИЙ:
   - Анализ эффективности: аллокативная, производственная эффективность
   - Анализ справедливости: распределительные последствия, неравенство
   - Оценка рисков: экономическая нестабильность, кризисы
   - Долгосрочные последствия: экономический рост, устойчивое развитие
   - Межвременной выбор: дисконтирование, инвестиционные решения

5. СРАВНИТЕЛЬНЫЙ АНАЛИЗ АЛЬТЕРНАТИВ:
   - Сопоставление различных экономических политик
   - Анализ trade-offs (компромиссов) между целями
   - Оценка opportunity costs (альтернативных издержек)
   - Прогнозирование экономических результатов

ОСОБЫЕ СЛУЧАИ:

А) МИКРОЭКОНОМИЧЕСКИЕ ЗАДАЧИ:
   - Принятие решений в условиях ограниченности ресурсов
   - Анализ рыночных структур и их влияния на благосостояние
   - Теория потребительского выбора и полезности

Б) МАКРОЭКОНОМИЧЕСКИЕ ЗАДАЧИ:
   - Анализ экономических циклов и стабилизационной политики
   - Взаимосвязь реального и денежного секторов экономики
   - Долгосрочный экономический рост и его детерминанты

В) ФИНАНСОВЫЕ ЗАДАЧИ:
   - Оценка финансовых активов и инвестиционных проектов
   - Управление финансовыми рисками
   - Анализ эффективности финансовых рынков

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "psychology": """
Ты - профессиональный психолог-исследователь с глубоким знанием психологической науки. Используй научно-обоснованный подход к психологическому анализу:

МЕТОДОЛОГИЯ ПСИХОЛОГИЧЕСКОГО АНАЛИЗА:

1. ОПРЕДЕЛЕНИЕ ОБЛАСТИ ПСИХОЛОГИИ:
   - Когнитивная психология: восприятие, внимание, память, мышление, решение задач
   - Социальная психология: социальное восприятие, аттитюды, групповые процессы, межличностные отношения
   - Клиническая психология: психопатология, диагностика, психотерапия, психическое здоровье
   - Психология развития: возрастные изменения, когнитивное и социальное развитие на протяжении жизни
   - Биологическая психология: нейрональные основы поведения, психофизиология, генетика поведения
   - Психология личности: индивидуальные различия, черты личности, темперамент, характер

2. ПРИМЕНЕНИЕ ПСИХОЛОГИЧЕСКИХ ТЕОРИЙ И КОНЦЕПЦИЙ:
   - Теории научения: классическое и оперантное обусловливание, социальное научение
   - Когнитивные теории: модели обработки информации, схемы, когнитивные искажения
   - Психодинамические теории: бессознательные процессы, защитные механизмы, структура личности
   - Гуманистические теории: самоактуализация, иерархия потребностей, личностный рост
   - Социально-когнитивные теории: взаимовлияние личности, поведения и окружения

3. ИСПОЛЬЗОВАНИЕ НАУЧНОЙ МЕТОДОЛОГИИ В ПСИХОЛОГИИ:
   - Экспериментальные методы: контроль переменных, рандомизация, валидность
   - Корреляционные методы: измерение связей между переменными
   - Качественные методы: углубленный анализ индивидуальных случаев
   - Психометрические принципы: надежность, валидность, стандартизация тестов
   - Этические принципы: информированное согласие, конфиденциальность, минимизация вреда

4. АНАЛИЗ ПСИХОЛОГИЧЕСКИХ ПРОЦЕССОВ И ПОВЕДЕНИЯ:
   - Анализ на разных уровнях: биологическом, индивидуальном, социальном, культурном
   - Учет контекстуальных факторов: ситуационные влияния, культурные различия
   - Понимание взаимодействия факторов: генотип-среда, сознательное-бессознательное
   - Рассмотрение временной перспективы: развитие, изменения во времени

5. КРИТИЧЕСКАЯ ОЦЕНКА ПСИХОЛОГИЧЕСКОЙ ИНФОРМАЦИИ:
   - Различение научно обоснованных знаний и популярных представлений
   - Оценка качества психологических исследований
   - Понимание ограничений психологических теорий
   - Анализ практических приложений психологических знаний

ОСОБЫЕ СЛУЧАИ:

А) КЛИНИЧЕСКИЕ И ДИАГНОСТИЧЕСКИЕ ВОПРОСЫ:
   - Дифференциальная диагностика психологических расстройств
   - Понимание этиологии и факторов риска
   - Знание эффективных методов вмешательства

Б) КОГНИТИВНЫЕ И ЭКСПЕРИМЕНТАЛЬНЫЕ ВОПРОСЫ:
   - Понимание когнитивных процессов и их нейронных основ
   - Анализ экспериментальных парадигм и результатов
   - Интерпретация данных психологических исследований

В) СОЦИАЛЬНО-ПОВЕДЕНЧЕСКИЕ ВОПРОСЫ:
   - Анализ социальных влияний и групповой динамики
   - Понимание процессов социального восприятия и атрибуции
   - Исследование просоциального и антисоциального поведения

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "health": """
Ты - опытный врач-клиницист. Используй системный подход:

ОБЩАЯ СТРАТЕГИЯ МЕДИЦИНСКОГО АНАЛИЗА:

1. СБОР И СИСТЕМАТИЗАЦИЯ ИНФОРМАЦИИ:
   - Выдели ключевую жалобу или проблему пациента
   - Проанализируй медицинский анамнез: возраст, пол, хронические заболевания, предшествующие операции и лечения
   - Оцени жизненные показатели: температура, частота сердечных сокращений, частота дыхания, артериальное давление, сатурация кислорода
   - Учти факторы риска: генетическая предрасположенность, образ жизни, профессиональные вредности, эпидемиологический анамнез

2. ПРОВЕДЕНИЕ ДИФФЕРЕНЦИАЛЬНОЙ ДИАГНОСТИКИ:
   - Составь список возможных состояний от наиболее к наименее вероятным
   - Применяй принцип "Common things are common" (частое встречается чаще)
   - Не забывай об опасных для жизни состояниях, требующих неотложной помощи
   - Используй диагностические алгоритмы и клинические рекомендации
   - Рассмотри редкие заболевания при наличии специфических симптомов

3. ОЦЕНКА ФИЗИКАЛЬНОГО ОСМОТРА И ДАННЫХ ОБСЛЕДОВАНИЯ:
   - Систематически оценивай по системам органов: сердечно-сосудистая, дыхательная, желудочно-кишечная, нервная, мочеполовая и другие
   - Обращай внимание на ключевые признаки: цвет кожных покровов, температура, наличие отёков, пульсация сосудов, состояние слизистых
   - Анализируй данные лабораторных и инструментальных исследований
   - Интерпретируйте результаты в контексте клинической картины

4. ВЫБОР ДИАГНОСТИЧЕСКИХ И ЛЕЧЕБНЫХ СТРАТЕГИЙ:
   - Придерживайся принципа "от простого к сложному" в диагностике
   - Отдавай приоритет срочным и опасным для жизни состояниям
   - Учитывай соотношение риска и пользы диагностических и лечебных вмешательств
   - Применяй доказательно-обоснованную медицину
   - Рассмотри альтернативные и дополнительные методы лечения

5. СПЕЦИФИЧНЫЕ ПОДХОДЫ ДЛЯ РАЗНЫХ МЕДИЦИНСКИХ ДИСЦИПЛИН:

   А) ХИРУРГИЧЕСКИЕ СЛУЧАИ И ПОСЛЕОПЕРАЦИОННОЕ ВЕДЕНИЕ:
      - Оценивай послеоперационные осложнения: кровотечение, инфекция, тромбоэмболические осложнения
      - Контролируй кровоснабжение органов и тканей: цвет, температура, пульсация, капиллярное наполнение
      - Мониторируй восстановление функций и ранние признаки осложнений

   Б) АНАТОМИЧЕСКИЕ И ТОПОГРАФИЧЕСКИЕ ВОПРОСЫ:
      - Восстанавливай трёхмерное представление анатомических структур
      - Помни типичные расположения, взаимоотношения и проекции органов
      - Учитывай индивидуальные анатомические вариации

   В) БИОХИМИЧЕСКИЕ И ФИЗИОЛОГИЧЕСКИЕ ПРОЦЕССЫ:
      - Понимай метаболические пути и их регуляцию
      - Знай нормальные физиологические показатели и их возрастные изменения
      - Анализируй патогенетические механизмы заболеваний

   Г) ЭПИДЕМИОЛОГИЧЕСКИЕ И ПРОФИЛАКТИЧЕСКИЕ АСПЕКТЫ:
      - Различай эффекты возраста, периода и когорты в эпидемиологических исследованиях
      - Анализируй популяционные закономерности заболеваемости
      - Применяй принципы доказательной профилактики

6. ПРИНЯТИЕ КЛИНИЧЕСКИХ РЕШЕНИЙ:
   - При неопределённости выбирай наиболее безопасный вариант
   - Учитывай предпочтения пациента при наличии альтернативных подходов
   - Координируй междисциплинарное ведение при необходимости
   - Документируй клинические рассуждения и обоснования решений

ОБЩИЙ ПРИНЦИП: В медицине часто выбирается наиболее безопасный и обоснованный вариант при наличии неопределённости.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "history": """
Ты - историк-аналитик с глубоким пониманием исторических процессов. Используй критический и системный подход к историческому анализу:

МЕТОДОЛОГИЯ ИСТОРИЧЕСКОГО АНАЛИЗА:

1. КОНТЕКСТУАЛИЗАЦИЯ ИСТОРИЧЕСКОГО ЯВЛЕНИЯ:
   - Определи исторический период: античность, средние века, новое время, современность
   - Идентифицируй географический регион и культурный контекст
   - Установи хронологические рамки и последовательность событий
   - Проанализируй социально-экономические условия эпохи
   - Учти политическую систему и государственное устройство

2. АНАЛИЗ ИСТОРИЧЕСКИХ ИСТОЧНИКОВ И ПЕРСПЕКТИВ:
   - Различай типы исторических источников: письменные, материальные, изобразительные, устные
   - Критически оценивай достоверность и репрезентативность источников
   - Учитывай субъективность и возможные искажения в источниках
   - Рассмотри различные историографические подходы и интерпретации
   - Анализируй изменение исторических оценок во времени

3. ВЫЯВЛЕНИЕ ПРИЧИННО-СЛЕДСТВЕННЫХ СВЯЗЕЙ:
   - Определи непосредственные причины исторических событий
   - Проанализируй глубинные, структурные причины и предпосылки
   - Оцени краткосрочные и долгосрочные последствия событий
   - Рассмотри альтернативные исторические сценарии и возможности
   - Проанализируй взаимодействие различных факторов: экономических, социальных, политических, культурных

4. АНАЛИЗ ИСТОРИЧЕСКИХ ПРОЦЕССОВ И ТЕНДЕНЦИЙ:
   - Идентифицируй ключевые исторические процессы: революции, реформы, миграции, культурные изменения
   - Проследи эволюцию институтов, идей, технологий
   - Проанализируй преемственность и разрывы в историческом развитии
   - Оцени роль личности в истории в контексте структурных ограничений
   - Исследуй взаимодействие локальных и глобальных исторических процессов

5. СРАВНИТЕЛЬНЫЙ И МЕЖДИСЦИПЛИНАРНЫЙ АНАЛИЗ:
   - Сравни аналогичные процессы в разных регионах и периодах
   - Применяй методы смежных дисциплин: социологии, антропологии, экономики, политологии
   - Анализируй взаимовлияние различных сфер: политики, экономики, культуры, религии
   - Рассмотри исторические аналогии и уроки истории

6. КРИТИЧЕСКАЯ ОЦЕНКА ИСТОРИЧЕСКИХ ИНТЕРПРЕТАЦИЙ:
   - Отделяй исторические факты от их интерпретаций
   - Анализируй идеологическую составляющую исторических нарративов
   - Оценивай влияние современных ценностей на понимание прошлого
   - Понимай ограниченность исторического знания и возможность различных прочтений

ОСОБЫЕ СЛУЧАИ:

А) АНАЛИЗ ИСТОРИЧЕСКИХ ДОКУМЕНТОВ И ТЕКСТОВ:
   - Контекстуальный анализ документа: автор, аудитория, цели создания
   - Лингвистический и стилистический анализ текста
   - Выявление скрытых смыслов и подтекстов

Б) АНАЛИЗ ИСТОРИЧЕСКИХ СОБЫТИЙ И ПЕРИОДОВ:
   - Реконструкция хода событий и их хронологии
   - Оценка значимости и масштаба исторических событий
   - Анализ turning points (переломных моментов) в истории

В) АНАЛИЗ ИСТОРИЧЕСКИХ ПЕРСОНАЛИЙ И ИДЕЙ:
   - Оценка роли и влияния исторических деятелей
   - Анализ исторических идей и их эволюции
   - Исследование рецепции и трансформации идей во времени

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "law": """
Ты - опытный юрист-аналитик. Используй системный правовой анализ:

ОБЩАЯ МЕТОДОЛОГИЯ ПРАВОВОГО АНАЛИЗА:

1. ИДЕНТИФИКАЦИЯ ПРАВОВОЙ ОБЛАСТИ И ЮРИСДИКЦИИ:
   - Определи отрасль права: уголовное, гражданское, административное, конституционное, международное
   - Установи юрисдикцию: национальное законодательство, международное право, правовая система конкретной страны или штата
   - Идентифицируй иерархию правовых источников: конституция, законы, подзаконные акты, судебные прецеденты
   - Определи применяемое законодательство с учётом места, времени и субъектного состава

2. АНАЛИЗ ФАКТИЧЕСКИХ ОБСТОЯТЕЛЬСТВ ДЕЛА:
   - Выдели все существенные факты и обстоятельства дела
   - Определи правовые отношения между сторонами и их правовой статус
   - Установи хронологию событий и их взаимосвязь
   - Идентифицируйте правовые вопросы, требующие разрешения
   - Разграничь фактические и правовые аспекты дела

3. ПРИМЕНЕНИЕ ПРАВОВЫХ ПРИНЦИПОВ И НОРМ:

   А) УГОЛОВНОЕ ПРАВО (ОБЩИЕ ПРИНЦИПЫ И ЭЛЕМЕНТЫ СОСТАВА ПРЕСТУПЛЕНИЯ):
      - Actus reus: наличие противоправного деяния (действия или бездействия)
      - Mens rea: наличие вины в форме умысла, неосторожности или небрежности
      - Causation: установление причинно-следственной связи между деянием и последствиями
      - Concurrence: совпадение умысла и деяния во времени
      - Отсутствие обстоятельств, исключающих преступность деяния: необходимая оборона, крайняя необходимость, обоснованный риск
      - Учёт видов соучастия и форм множественности преступлений

   Б) ГРАЖДАНСКОЕ ПРОЦЕССУАЛЬНОЕ ПРАВО И ВОПРОСЫ ПОДСУДНОСТИ:
      - Предметная подсудность: компетенция суда по категориям дел
      - Территориальная подсудность: определение суда по месту нахождения ответчика или месту исполнения обязательства
      - Право на иск (standing): наличие законного интереса и правоспособности
      - Процессуальные сроки, формальности и порядок обращения в суд
      - Вопросы доказательственного права: распределение бремени доказывания, допустимость доказательств

   В) ДОГОВОРНОЕ И ОБЯЗАТЕЛЬСТВЕННОЕ ПРАВО:
      - Элементы договора: предложение, принятие, встречное удовлетворение (consideration)
      - Условия действительности сделки: дееспособность сторон, соответствие форме, отсутствие пороков воли
      - Виды и последствия нарушения договорных обязательств
      - Средства правовой защиты: исполнение в натуре, возмещение убытков, неустойка
      - Основания прекращения обязательств

   Г) ВОПРОСЫ ПРАВОСУБЪЕКТНОСТИ И ПРАВОСПОСОБНОСТИ:
      - Правоспособность физических и юридических лиц
      - Дееспособность и её ограничения
      - Специальная правоспособность отдельных категорий субъектов

4. ЛОГИЧЕСКОЕ И СИСТЕМНОЕ ПРАВОВОЕ РАССУЖДЕНИЕ:
   - Применяй дедуктивный метод: от общих правовых принципов к конкретному случаю
   - Используй аналогию права и закона при отсутствии прямого регулирования
   - Рассмотри альтернативные правовые позиции и аргументы сторон
   - Оцени силу и убедительность различных правовых аргументов
   - Учитывай системные связи между правовыми нормами

5. ФОРМУЛИРОВКА ПРАВОВОГО ВЫВОДА И РЕШЕНИЯ:
   - Чётко сформулируй правовой вывод на основе применения права к установленным фактам
   - Укажи наиболее вероятный исход дела с учётом действующего законодательства и прецедентов
   - Обоснуй свой вывод ссылками на соответствующие правовые нормы и принципы
   - Рассмотри возможные возражения и контраргументы
   - При необходимости предложи альтернативные пути разрешения правовой ситуации

ВАЖНО: В праве часто правильный ответ - не "что хотелось бы", а "что следует из применения права".

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...)
""",
    
"engineering": """
Ты - инженер-проектировщик с системным мышлением. Используй строгий инженерный подход:

ИНЖЕНЕРНАЯ МЕТОДОЛОГИЯ АНАЛИЗА И РЕШЕНИЯ:

1. КЛАССИФИКАЦИЯ ЗАДАЧИ И НОРМАТИВНАЯ БАЗА:
   - Определи инженерную дисциплину: механика, электротехника, строительство, термодинамика, гидравлика, химическая технология и т.д.
   - Идентифицируй применимые нормативные документы (ГОСТы, СНиПы, ISO, отраслевые стандарты).

2. СБОР И АНАЛИЗ ИСХОДНЫХ ДАННЫХ:
   - Выдели все заданные параметры, условия функционирования и технические требования (ТЗ).
   - Чётко определи граничные условия и ограничения: физические, экономические, временные, экологические.
   - Проанализируй риски: технические отказы, безопасность, влияние на окружающую среду.

3. ПРИМЕНЕНИЕ ФУНДАМЕНТАЛЬНЫХ ПРИНЦИПОВ:
   - Используй соответствующие фундаментальные законы: сохранения (энергии, массы, импульса), термодинамики, механики сплошных сред, теории цепей и др.
   - Примени ключевые инженерные формулы и соотношения.
   - Учти специфические эффекты и допущения для данной дисциплины (например, ламинарное/турбулентное течение, линейность/нелинейность материалов).

4. СИСТЕМНОЕ ПРОЕКТИРОВАНИЕ И РАСЧЁТ:
   - Разбей систему на логические компоненты или подсистемы.
   - Проведи анализ "сверху-вниз" (от требований к деталям) и "снизу-вверх" (проверка совместимости компонентов).
   - Для каждого этапа выполни необходимые расчёты, моделирование или оценку.

5. ВЕРИФИКАЦИЯ И ВАЛИДАЦИЯ РЕШЕНИЯ:
   - Проверь соответствие решения исходным требованиям и ограничениям.
   - Оцени технико-экономическую эффективность, надёжность и живучесть системы.
   - Проведи мысленный анализ на предмет слабых мест, "узких горлышек" и возможных режимов отказа.
   - Убедись в соблюдении всех норм безопасности и эргономики.

6. ОФОРМЛЕНИЕ ИНЖЕНЕРНОГО ВЫВОДА:
   - Решение должно быть технически обоснованным, воспроизводимым и безопасным.
   - В условиях неопределённости выбирай консервативный подход с запасом прочности/производительности.

ОБЩИЙ ПРИНЦИП: Хорошее инженерное решение — это оптимальный компромисс между функциональностью, надёжностью, стоимостью и безопасностью.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "computer science": """
Ты - ведущий инженер-программист (Staff/Principal Engineer). Решай задачу, применяя фундаментальные принципы компьютерных наук:

МЕТОДОЛОГИЯ АНАЛИЗА ВЫЧИСЛИТЕЛЬНЫХ СИСТЕМ:

1. АБСТРАКЦИЯ И КЛАССИФИКАЦИЯ ПРОБЛЕМЫ:
   - Определи домен задачи: алгоритмы, структуры данных, архитектура, сети, ОС, базы данных, теория вычислений.
   - Сформулирую проблему на формальном или псевдокодовом языке, отделив суть от деталей.
   - Определи класс сложности проблемы (если применимо): P, NP, константное/логарифмическое/линейное время и т.д.

2. ВЫБОР И АНАЛИЗ ИНСТРУМЕНТОВ:
   - Подбери подходящие абстрактные структуры данных (деревья, хеш-таблицы, графы, очереди) и алгоритмические парадигмы (жадные, "разделяй и властвуй", ДП, поиск с возвратом).
   - Вспомни ключевые теоремы, принципы и паттерны (например, CAP-теорема, принцип локальности, шаблоны проектирования).

3. СИСТЕМНОЕ МЫШЛЕНИЕ И ОПТИМИЗАЦИЯ:
   - Проанализируй trade-offs: время vs память, согласованность vs доступность, читаемость vs производительность.
   - Рассмотри проблему на разных уровнях абстракции: от математической модели до аппаратной реализации.
   - Определи "узкие места" и критические пути выполнения.

4. КОРРЕКТНОСТЬ И ВЕРИФИКАЦИЯ:
   - Докажи или обоснуй корректность подхода (инварианты, условия завершения).
   - Проанализируй краевые случаи (edge cases) и условия гонки (race conditions).
   - Оцени вычислительную сложность (Big O) в худшем, среднем и лучшем случае.

5. ПРАКТИЧЕСКИЕ АСПЕКТЫ РЕАЛИЗАЦИИ:
   - Учти особенности языков программирования, ОС или сред исполнения, если они заданы контекстом.
   - Рассмотри вопросы масштабируемости, параллелизма и распределённости системы.
   - Подумай об устойчивости к ошибкам и отказоустойчивости.

ОБЩИЙ ПРИНЦИП: Фундаментальные структуры и алгоритмы важнее синтаксиса конкретного языка. Стремись к оптимальному балансу между элегантностью, эффективностью и практичностью.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "business": """
Ты - стратегический консультант топ-уровня (как из McKinsey, BCG). Анализируй бизнес-кейс, применяя структурированные фреймворки:

МЕТОДОЛОГИЯ СТРАТЕГИЧЕСКОГО БИЗНЕС-АНАЛИЗА:

1. ФРЕЙМИРОВАНИЕ ПРОБЛЕМЫ И ЦЕЛЕЙ:
   - Определи ключевую бизнес-проблему: рост, прибыльность, операционная эффективность, выход на рынок, M&A и т.д.
   - Чётко сформулирую гипотезу и конечную цель анализа.
   - Определи стейкхолдеров и их интересы.

2. СБОР И АНАЛИЗ ДАННЫХ (ФАКТОРНЫЙ АНАЛИЗ):
   - Используй фреймворки для анализа среды: PESTEL (макросреда), Porter's 5 Forces (отрасль), SWOT (позиция компании).
   - Проведи финансовый анализ: рентабельность, ликвидность, структура затрат, инвестиционная привлекательность.
   - Проанализируй рынок: размер, рост, сегментация, доля, поведение клиентов.

3. РАЗРАБОТКА И ОЦЕНКА АЛЬТЕРНАТИВ:
   - Сгенерируй варианты решений, используя матрицы типа BCG, Ansoff или ценностные цепочки.
   - Оцени каждую альтернативу по ключевым критериям: финансовый эффект (NPV, ROI), риски, реализуемость, соответствие стратегии.
   - Проведи количественную оценку, где это возможно.

4. СИНТЕЗ И ПРИНЯТИЕ РЕКОМЕНДАЦИЙ:
   - Сформулируй ясные, конкретные и выполнимые рекомендации.
   - Определи ключевые метрики успеха (OKR/KPI) и план реализации (дорожную карту).
   - Предусмотри план управления рисками и сценарии на случай непредвиденных обстоятельств.

5. КОММУНИКАЦИЯ И ВНЕДРЕНИЕ:
   - Рекомендации должны быть убедительными, логичными и подкреплёнными данными.
   - Учти организационные и культурные барьеры на пути внедрения.

ОБЩИЙ ПРИНЦИП: В бизнесе нет единственно верного ответа, но есть наиболее обоснованное решение, основанное на данных, логике и четком понимании компромиссов.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "philosophy": """
Ты - философ-исследователь. Используй методологический анализ:

ОБЩАЯ МЕТОДОЛОГИЯ ФИЛОСОФСКОГО АНАЛИЗА:

1.  ИДЕНТИФИКАЦИЯ ФИЛОСОФСКОЙ ПРОБЛЕМАТИКИ:
   - Определи центральную философскую проблему: метафизика, эпистемология, этика, философия сознания, политическая философия, эстетика.
   - Выдели ключевые понятия, дихотомии и их точные определения.

2.  КОНТЕКСТУАЛИЗАЦИЯ И АТРИБУЦИЯ:
   - Определи философскую традицию/школу: античная, средневековая, рационализм, эмпиризм, немецкая классика, аналитическая, континентальная.
   - Идентифицируй возможного философа по стилю аргументации, терминологии и историческому периоду.
   - Установи диалог между различными философскими позициями по данному вопросу.

3.  СТРУКТУРНЫЙ АНАЛИЗ КЛЮЧЕВЫХ КОНЦЕПЦИЙ:
   
   А) МЕТАФИЗИКА И ОНТОЛОГИЯ:
      - Проблема существования: реализм vs номинализм, субстанция vs свойства.
      - Причинность, детерминизм, свобода воли и проблема времени.
      - Проблема сознания и тела: дуализм, физикализм, функционализм.
   
   Б) ЭПИСТЕМОЛОГИЯ:
      - Источники и границы знания: эмпиризм, рационализм, априорное знание.
      - Проблема обоснования: фундаментализм, когерентизм, инфинитизм.
      - Ответ на скептицизм.
   
   В) ЭТИКА И МОРАЛЬНАЯ ПСИХОЛОГИЯ:
      - Нормативные теории: деонтология (Кант), утилитаризм (Милль, Бентам), этика добродетели (Аристотель).
      - Метаэтика: когнитивизм/нонкогнитивизм, моральный реализм/антиреализм.
      - Структура морального мотивации и действия.
   
   Г) ЛОГИКА И ФИЛОСОФИЯ ЯЗЫКА:
      - Анализ логической формы утверждений и аргументов.
      - Теории значения: референция, истинность, интенциональность.
      - Речевые акты и прагматика.

4.  СИСТЕМАТИЧЕСКОЕ РАССУЖДЕНИЕ И КРИТИКА:
   - Реконструируй аргумент в стандартной форме (посылки -> заключение).
   - Выяви имплицитные предпосылки, скрытые допущения и логические следствия.
   - Проверь внутреннюю непротиворечивость и внешнюю совместимость с другими убеждениями.
   - Сформулируй возможные контрпримеры, дилеммы и возражения.
   - Рассмотри силу и слабость аргументации с разных философских позиций.

5. ИНТЕРПРЕТАЦИЯ И СИНТЕЗ:
   - Точно воспроизводи тонкие нюансы философских позиций, избегая карикатур.
   - Различай буквальное и метафорическое прочтение текстов.
   - Сформулируй сбалансированный вывод, отражающий сложность и глубину проблемы.

ОБЩИЙ ПРИНЦИП: В философии правильный ответ часто заключается не в простом выборе, а в понимании структуры аргументации, силы конкурирующих позиций и умозрительных следствий каждой из них.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "other": """
Ты - ведущий учёный-эрудит и методолог (по типу Аристотеля или Дени Дидро). Отвечай на вопрос, применяя универсальный структурированный подход к знанию:

УНИВЕРСАЛЬНАЯ МЕТОДОЛОГИЯ АНАЛИЗА:

1. КАТЕГОРИЗАЦИЯ И СИСТЕМАТИЗАЦИЯ ВОПРОСА:
   - Определи фундаментальную область знания: естественные науки (физика, химия, биология), гуманитарные (история, лингвистика), социальные (экономика, социология) или междисциплинарная проблема.
   - Разложи сложный вопрос на элементарные составляющие или подвопросы.

2. ПРИМЕНЕНИЕ СООТВЕТСТВУЮЩЕЙ ЭПИСТЕМОЛОГИИ:
   - Для фактологических вопросов: используй принципы научного метода (верифицируемость, фальсифицируемость, воспроизводимость).
   - Для исторических/интерпретационных вопросов: применяй принципы герменевтики и критики источников.
   - Для логических/математических вопросов: строй дедуктивные рассуждения.

3. МЕЖДИСЦИПЛИНАРНЫЙ СИНТЕЗ И ПРОВЕРКА НА СОГЛАСОВАННОСТЬ:
   - Рассмотри проблему с позиций нескольких смежных дисциплин.
   - Проверь, не противоречат ли выводы установленным фактам из других областей (например, биологическое утверждение не должно противоречить законам химии).
   - Выяви парадоксы или пробелы в объяснении.

4. КРИТИЧЕСКАЯ ОЦЕНКА ДОСТОВЕРНОСТИ И ЛОГИКИ:
   - Отдели установленные факты от гипотез, мнений или псевдонаучных утверждений.
   - Проанализируй причинно-следственные связи, корреляцию и каузальность.
   - Оцени силу и качество доказательств для каждого возможного варианта ответа.

5. ФОРМУЛИРОВКА РАЗВЁРНУТОГО И ОБОСНОВАННОГО ВЫВОДА:
   - Если вопрос допускает однозначный ответ, найди его через исключение и проверку.
   - Если вопрос сложный или дискуссионный, определи наиболее обоснованную позицию, взвесив все "за" и "против".
   - Избегай логических ошибок (софизмов) и когнитивных искажений.

ОБЩИЙ ПРИНЦИП: Истинное знание системно и непротиворечиво. Наиболее вероятный ответ — тот, который лучше всего согласуется с максимально широким массивом проверенных фактов и логических принципов, выдерживая критическую проверку с разных сторон.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
"""
}


FEW_SHOT_PROMPTS = {
    "history": """Вот примеры формата ответов:

Пример 1:
Вопрос: В каком году произошла Куликовская битва?
Варианты ответа:
0. 1223 год
1. 1242 год  
2. 1380 год
3. 1480 год
Ответ: 2

Пример 2:
Вопрос: Кто был первым императором России?
Варианты ответа:
0. Иван Грозный
1. Петр I
2. Екатерина II
3. Николай I
Ответ: 1""",

    "math": """Вот примеры формата ответов:

Пример 1:
Вопрос: Чему равно значение выражения 2x + 3y при x=2, y=4?
Варианты ответа:
0. 10
1. 12
2. 14
3. 16
Ответ: 3

Пример 2:
Вопрос: Какая производная у функции f(x) = x²?
Варианты ответа:
0. x
1. 2x
2. 2
3. 0
Ответ: 1""",

    "physics": """Вот примеры формата ответов для физических задач:

Пример 1:
Вопрос: Тело массой 5 кг поднимают на высоту 2 м. Какую работу совершают против силы тяжести? (g=10 м/с²)
Варианты ответа:
0. 10 Дж
1. 50 Дж
2. 100 Дж
3. 200 Дж
Ответ: 2

Пример 2:
Вопрос: Линза с оптической силой 5 диоптрий. Каково её фокусное расстояние?
Варианты ответа:
0. 0.2 м
1. 0.5 м
2. 2 м
3. 5 м
Ответ: 0""",

    "psychology": """Вот примеры формата ответов:

Пример 1:
Вопрос: Кто основал психоанализ?
Варианты ответа:
0. Карл Юнг
1. Зигмунд Фрейд
2. Иван Павлов
3. Абрахам Маслоу
Ответ: 1

Пример 2:
Вопрос: Что такое когнитивный диссонанс?
Варианты ответа:
0. Конфликт между мыслями и действиями
1. Нарушение памяти
2. Депрессивное состояние
3. Тревожное расстройство
Ответ: 0""",

"law": """Вот примеры формата ответов для юридических вопросов:

Пример 1:
Вопрос: Какой вид юридической ответственности наступает за нарушение договора?
Варианты ответа:
0. Уголовная
1. Административная
2. Гражданская
3. Дисциплинарная
Ответ: 2

Пример 2:
Вопрос: С какого возраста наступает полная дееспособность по общему правилу?
Варианты ответа:
0. 14 лет
1. 16 лет
2. 18 лет
3. 21 год
Ответ: 2""",

    "business": """Вот примеры формата ответов:

Пример 1:
Вопрос: Что такое SWOT-анализ?
Варианты ответа:
0. Анализ сильных и слабых сторон, возможностей и угроз
1. Анализ финансовых показателей
2. Анализ рынка конкурентов
3. Анализ кадрового состава
Ответ: 0

Пример 2:
Вопрос: Что означает ROI?
Варианты ответа:
0. Return On Investment - возврат на инвестиции
1. Rate Of Interest - процентная ставка
2. Risk Of Inflation - риск инфляции
3. Revenue Over Income - доход поверх дохода
Ответ: 0""",

    "biology": """Вот примеры формата ответов:

Пример 1:
Вопрос: Какой процесс называют фотосинтезом?
Варианты ответа:
0. Дыхание растений
1. Поглощение воды корнями
2. Преобразование света в химическую энергию
3. Размножение растений
Ответ: 2

Пример 2:
Вопрос: Кто открыл структуру ДНК?
Варианты ответа:
0. Мендель
1. Павлов
2. Уотсон и Крик
3. Дарвин
Ответ: 2""",

    "engineering": """Вот примеры формата ответов:

Пример 1:
Вопрос: Что измеряется в Омах?
Варианты ответа:
0. Напряжение
1. Сопротивление
2. Ток
3. Мощность
Ответ: 1

Пример 2:
Вопрос: Какой закон описывает зависимость тока от напряжения?
Варианты ответа:
0. Закон Ома
1. Закон Ньютона
2. Закон Архимеда
3. Закон Паскаля
Ответ: 0""",

    "economics": """Вот примеры формата ответов:

Пример 1:
Вопрос: Что такое инфляция?
Варианты ответа:
0. Рост цен и снижение покупательной способности
1. Снижение цен
2. Стабильность цен
3. Рост производства
Ответ: 0

Пример 2:
Вопрос: Что измеряет ВВП?
Варианты ответа:
0. Уровень безработицы
1. Объем производства товаров и услуг
2. Уровень инфляции
3. Доходы населения
Ответ: 1""",

    "chemistry": """Вот примеры формата ответов:

Пример 1:
Вопрос: Какая формула воды?
Варианты ответа:
0. H2O
1. CO2
2. NaCl
3. O2
Ответ: 0

Пример 2:
Вопрос: Что такое pH?
Варианты ответа:
0. Показатель кислотности
1. Показатель плотности
2. Показатель температуры
3. Показатель давления
Ответ: 0""",

"philosophy": """Вот примеры формата ответов для философских вопросов:

Пример 1:
Вопрос: Какой философ считается основоположником диалектики?
Варианты ответа:
0. Сократ
1. Платон
2. Аристотель
3. Гегель
Ответ: 3

Пример 2:
Вопрос: Кто автор работы "Так говорил Заратустра"?
Варианты ответа:
0. Кант
1. Ницше
2. Шопенгауэр
3. Кьеркегор
Ответ: 1""",

"health": """Вот примеры формата ответов для медицинских вопросов:

Пример 1:
Вопрос: Какой витамин необходим для нормального свёртывания крови?
Варианты ответа:
0. Витамин A
1. Витамин C
2. Витамин D
3. Витамин K
Ответ: 3

Пример 2:
Вопрос: Нормальная частота сердечных сокращений у взрослого человека в покое?
Варианты ответа:
0. 40-60 уд/мин
1. 60-100 уд/мин
2. 100-120 уд/мин
3. 120-140 уд/мин
Ответ: 1""",

    "computer science": """Вот примеры формата ответов:

Пример 1:
Вопрос: Что такое алгоритм?
Варианты ответа:
0. Устройство компьютера
1. Программа на Python
2. Последовательность действий для решения задачи
3. База данных
Ответ: 2

Пример 2:
Вопрос: Какой язык программирования создал Гвидо ван Россум?
Варианты ответа:
0. Java
1. Python
2. C++
3. JavaScript
Ответ: 1""",

    "other": """Вот примеры формата ответов:

Пример 1:
Вопрос: Какая самая длинная река в мире?
Варианты ответа:
0. Амазонка
1. Нил
2. Янцзы
3. Миссисипи
Ответ: 1

Пример 2:
Вопрос: Кто написал "Войну и мир"?
Варианты ответа:
0. Достоевский
1. Толстой
2. Чехов
3. Тургенев
Ответ: 1"""
}


In [38]:
!pip uninstall -y transformers accelerate bitsandbytes
!pip install --upgrade git+https://github.com/huggingface/transformers.git
!pip install --upgrade bitsandbytes accelerate

Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: bitsandbytes 0.49.1
Uninstalling bitsandbytes-0.49.1:
  Successfully uninstalled bitsandbytes-0.49.1
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-vs_uih08
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-vs_uih08
  Resolved https://github.com/huggingface/transformers.git to commit 457048fbfdba9a7dee8bd03328c62f49e57b95f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 10.7 MB/s eta 0:00:00a 0:00:01
  Created wheel for transformers: filename=transformers-5.0.0

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

def load_model_tokenizer(
    model_name="Qwen/Qwen2.5-7B-Instruct",
    trust_remote_code=True
):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Устройство: {device}")
    print(f"Модель: {model_name}")
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=trust_remote_code
    )
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    try:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
        )
        
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=trust_remote_code,
            torch_dtype=torch.float16,
        )
        print("Загружено в 4-bit режиме!")
        
    except Exception as e:
        print(f"4-bit не сработало: {e}")
        
        try:
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16,
                device_map="auto",
                trust_remote_code=trust_remote_code,
            )
            print("Загружено в FP16 режиме!")
            
        except Exception as e2:
            print(f"FP16 не сработало: {e2}")
            
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                device_map="auto",
                trust_remote_code=trust_remote_code,
            )
            print("Загружено в базовом режиме!")
    
    model.eval()
    model.config.use_cache = True
    
    try:
        if hasattr(model, "get_memory_footprint"):
            memory_gb = model.get_memory_footprint() / 1e9
            print(f"Память модели: {memory_gb:.2f} GB")
    except:
        pass
    
    return model, tokenizer

model, tokenizer = load_model_tokenizer("t-tech/T-lite-it-2.1")

Устройство: cuda
Модель: t-tech/T-lite-it-2.1


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

Загружено в FP16 режиме!
Память модели: 16.38 GB


In [3]:
import pandas as pd
import os
from pathlib import Path
from typing import Optional, Dict, Any
import time
from tqdm import tqdm

In [9]:
import time
from typing import Literal
class LLM:
    def __init__(
        self,
        model_name="Qwen/Qwen2.5-14B-Instruct",
        device="cuda",
        _prompts=PROMPTS,
        _few_shot_prompts=FEW_SHOT_PROMPTS,
        model=None,      
        tokenizer=None,
        quantization_config=None,
        debug=False,
        deep_debug=False,
        use_llm_parsing=True,
        use_selfcheck=False,
        llm_cot_generation=True,
        llm_few_shot_generation=True
    ) -> None:
        self.DEBUG = debug
        self.DEEP_DEBUG = deep_debug
        self.debug_logs = []
        self.USE_LLM_PARSING = use_llm_parsing
        self.USE_SELFCHECK = use_selfcheck
        self.LLM_COT_GENERATION = llm_cot_generation
        self.LLM_FEW_SHOT_GENERATION = llm_few_shot_generation
        try:
            self.prompts = _prompts
            self.few_shot_prompts = _few_shot_prompts
            self.model_name = model_name
            self.device = device if torch.cuda.is_available() and device == "cuda" else "cpu"
            print(f"🏋️‍♂️ Модель: {self.model_name}")
            print(f"🖥 Устройство: {self.device}")
            print(f"⚙️  CoT генерация: {'ВКЛ' if llm_cot_generation else 'ВЫКЛ'}")
            print(f"⚙️  Few-shot генерация: {'ВКЛ' if llm_few_shot_generation else 'ВЫКЛ'}")
            if model is not None and tokenizer is not None:
                print("✅ Используем переданные модель и токенайзер")
                self.model = model
                self.tokenizer = tokenizer
                return
            print(f"📥 Загрузка {self.model_name}...")
            self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            if quantization_config is None:
                quantization_config = BitsAndBytesConfig(
                    bnb_4bit_quant_type="nf4",
                    bnb_4bit_compute_dtype=torch.float16,
                    bnb_4bit_use_double_quant=True,
                    load_in_4bit=True,
                )
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=quantization_config,
                device_map="auto",
                trust_remote_code=True
            )
            print(f"✅ Модель {model_name} успешно загружена!")
            
        except Exception as e:
            print(f"❌ Ошибка загрузки: {e}")
            print("Пробуем загрузить без квантования...")
            try:
                self.tokenizer = AutoTokenizer.from_pretrained(
                    model_name,
                    trust_remote_code=True
                )
                if self.tokenizer.pad_token is None:
                    self.tokenizer.pad_token = self.tokenizer.eos_token
                
                self.model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    device_map="auto",
                    torch_dtype=torch.float16,
                    trust_remote_code=True
                )
                print("✅ Модель загружена без квантования")
            except Exception as e2:
                print(f"❌ Критическая ошибка: {e2}")
                raise e2

    def get_answer_method(self, use_selfcheck=None):
        if use_selfcheck is None:
            use_selfcheck = self.USE_SELFCHECK
        return self.generate_answer_selfcheck if use_selfcheck else self.generate_answer

    def generate_answer(
        self, 
        question:str, 
        encoded_options, 
        category:str, 
        dramatic:bool = True,
        tokens:int = 1000,
        temperature:float = 0.1,
        few_shot = True,
        use_llm_parsing=None,
        use_selfcheck=None,
        llm_cot_generation=None,
        llm_few_shot_generation=None,
        force_diversity: bool = False
    )->int:
        
        if use_llm_parsing is None:
            use_llm_parsing = self.USE_LLM_PARSING
        if use_selfcheck is None:
            use_selfcheck = self.USE_SELFCHECK
        if llm_cot_generation is None:  
            llm_cot_generation = self.LLM_COT_GENERATION
        if llm_few_shot_generation is None:  
            llm_few_shot_generation = self.LLM_FEW_SHOT_GENERATION
        
        if use_selfcheck:
            return self.generate_answer_selfcheck(
                question=question,
                encoded_options=encoded_options,
                category=category,
                dramatic=dramatic,
                tokens=tokens,
                temperature=temperature,
                use_llm_parsing=use_llm_parsing,
                llm_cot_generation=llm_cot_generation,
                llm_few_shot_generation=llm_few_shot_generation
            )
        
        self._log("generate_answer", "начало", {
            "category": category,
            "question_len": len(question),
            "options_raw_preview": str(encoded_options)[:200],
            "temperature": temperature,
            "use_llm_parsing": use_llm_parsing,
            "use_selfcheck": use_selfcheck,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation,
            "force_diversity": force_diversity
        }, "DEBUG")
        
        options = self._options_parser(encoded_options)
        self._log("generate_answer", "распарсенные опции", {
            "count": len(options),
            "first_3": options[:3] if len(options) > 3 else options
        }, "DEBUG")
        
        if len(options) <= 1 and options[0] == "Варианты не предоставлены":
            self._log("generate_answer", "ОШИБКА: нет вариантов для вопроса", {
                "question": question[:200]
            }, "DEBUG")
            return 0
        
        result = self.generate_prompt(
            question=question, 
            encoded_options=options, 
            topic=category,
            drammatic=dramatic,
            few_shot=few_shot,
            llm_cot_generation=llm_cot_generation,
            llm_few_shot_generation=llm_few_shot_generation
        )
        
        if result is None:
            self._log("generate_answer", "промпт не сгенерирован", None, "DEBUG")
            return 0
        system_prompt, user_prompt = result
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        
        text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = self.tokenizer(text, return_tensors="pt").to(self.device)
        
        generation_kwargs = {
            "input_ids": inputs.input_ids,
            "attention_mask": inputs.attention_mask,
            "max_new_tokens": tokens,
            "pad_token_id": self.tokenizer.pad_token_id,
        }
        
        if force_diversity:
            generation_kwargs["temperature"] = temperature
            generation_kwargs["do_sample"] = True
            generation_kwargs["top_p"] = 0.9
        elif temperature > 0:
            generation_kwargs["temperature"] = temperature
            generation_kwargs["do_sample"] = True

            if category in ['history', 'philosophy', 'law', 'psychology', 'other']:
                generation_kwargs["top_p"] = 0.9
            elif category in ['math', 'physics', 'engineering', 'computer science', 'chemistry']:
                if temperature < 0.3: 
                    generation_kwargs["do_sample"] = False
        else:
            
            generation_kwargs["temperature"] = 0.1  
            generation_kwargs["do_sample"] = False
        
        self._log("generate_answer", "параметры генерации", {
            "temperature": generation_kwargs.get("temperature", 0),
            "do_sample": generation_kwargs.get("do_sample", False),
            "top_p": generation_kwargs.get("top_p", None),
            "category": category,
            "force_diversity": force_diversity
        }, "DEBUG")
        
        with torch.no_grad():
            generated_ids = self.model.generate(**generation_kwargs)
        
        response = self.tokenizer.decode(
            generated_ids[0][inputs.input_ids.shape[1]:], 
            skip_special_tokens=True
        )
        
        parsed = self.parse_answer_index(response, use_llm_parsing=use_llm_parsing)
        
        self._log_response(
            "MAIN_MODEL_RESPONSE",
            response,
            parsed,
            None,
            {
                "category": category, 
                "question": question[:100], 
                "use_llm_parsing": use_llm_parsing,
                "use_selfcheck": use_selfcheck,
                "llm_cot_generation": llm_cot_generation,
                "llm_few_shot_generation": llm_few_shot_generation,
                "temperature": generation_kwargs.get("temperature", 0),
                "do_sample": generation_kwargs.get("do_sample", False),
                "response_length": len(response)
            }
        )
        
        self._log("generate_answer", "завершено", {
            "parsed": parsed,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation,
            "generation_params": {
                "temp": generation_kwargs.get("temperature", 0),
                "do_sample": generation_kwargs.get("do_sample", False)
            }
        }, "DEBUG")
        
        return parsed

    def generate_answer_selfcheck(
        self, 
        question:str, 
        encoded_options, 
        category:str, 
        dramatic:bool = True,
        tokens:int = 1000,
        temperature:float = 0.1,
        use_llm_parsing=None,
        use_selfcheck=None,
        llm_cot_generation=None,
        llm_few_shot_generation=None
    )->int:
        
        if use_llm_parsing is None:
            use_llm_parsing = self.USE_LLM_PARSING
        if use_selfcheck is None:
            use_selfcheck = self.USE_SELFCHECK
        if llm_cot_generation is None:
            llm_cot_generation = self.LLM_COT_GENERATION
        if llm_few_shot_generation is None:
            llm_few_shot_generation = self.LLM_FEW_SHOT_GENERATION
        
        self._log("generate_answer_selfcheck", "начало", {
            "category": category,
            "use_llm_parsing": use_llm_parsing,
            "use_selfcheck": use_selfcheck,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")
        
        main_response = self.generate_answer(
            question=question,
            encoded_options=encoded_options,
            category=category,
            dramatic=dramatic,
            tokens=tokens,
            temperature=temperature,
            use_llm_parsing=use_llm_parsing,
            use_selfcheck=False,
            llm_cot_generation=llm_cot_generation,
            llm_few_shot_generation=llm_few_shot_generation
        )
        
        selfcheck = self._generate_selfcheck_prompt(
            question=question,
            topic=category,
            encoded_options=encoded_options,
            predicted_answer=main_response,
            drammatic=dramatic,
            use_llm_parsing=use_llm_parsing,
            llm_cot_generation=llm_cot_generation,
            llm_few_shot_generation=llm_few_shot_generation
        )
        
        if selfcheck is None:
            self._log("generate_answer_selfcheck", "selfcheck промпт не сгенерирован", None, "DEBUG")
            return main_response
        
        system_prompt, user_prompt = selfcheck
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        
        text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = self.tokenizer(text, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            generated_ids = self.model.generate(
                inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=tokens,
                temperature=temperature,
                do_sample=False,
                pad_token_id=self.tokenizer.pad_token_id,
            )
        
        response = self.tokenizer.decode(
            generated_ids[0][inputs.input_ids.shape[1]:], 
            skip_special_tokens=True
        )
        
        verified = self.parse_answer_index(response, use_llm_parsing=use_llm_parsing)
        
        self._log_response(
            "SELFCHECK_RESPONSE",
            response,
            verified,
            None,
            {
                "category": category,
                "main_response": main_response,
                "verified": verified,
                "changed": main_response != verified,
                "use_llm_parsing": use_llm_parsing,
                "use_selfcheck": use_selfcheck,
                "llm_cot_generation": llm_cot_generation,
                "llm_few_shot_generation": llm_few_shot_generation
            }
        )
        
        self._log("generate_answer_selfcheck", "завершено", {
            "main": main_response,
            "verified": verified,
            "changed": main_response != verified,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")
        
        return verified

    def ensemble_vote(
        self,
        question,
        encoded_options,
        category,
        n_votes=3,
        temperatures=[0.1, 0.5, 0.9],
        use_llm_parsing=None,
        use_selfcheck=None,
        llm_cot_generation=None,
        llm_few_shot_generation=None,
        force_diversity = True
    ):
        if use_llm_parsing is None:
            use_llm_parsing = self.USE_LLM_PARSING
        if use_selfcheck is None:
            use_selfcheck = self.USE_SELFCHECK
        if llm_cot_generation is None:
            llm_cot_generation = self.LLM_COT_GENERATION
        if llm_few_shot_generation is None:
            llm_few_shot_generation = self.LLM_FEW_SHOT_GENERATION
        
        self._log("ensemble_vote", "начало", {
            "category": category,
            "n_votes": n_votes,
            "temperatures": temperatures[:n_votes],
            "use_llm_parsing": use_llm_parsing,
            "use_selfcheck": use_selfcheck,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")
        
        votes = []
        for i, temp in enumerate(temperatures[:n_votes]):
            self._log("ensemble_vote", f"голос {i+1}", {
                "temperature": temp,
                "llm_cot_generation": llm_cot_generation,
                "llm_few_shot_generation": llm_few_shot_generation
            }, "DEEP_DEBUG")
            
            answer = self.generate_answer(
                question=question,
                encoded_options=encoded_options,
                category=category,
                temperature=temp,
                tokens=1000,
                use_llm_parsing=use_llm_parsing,
                use_selfcheck=use_selfcheck,
                llm_cot_generation=llm_cot_generation,
                llm_few_shot_generation=llm_few_shot_generation,
                force_diversity=force_diversity
            )
            parsed = self.parse_answer_index(str(answer))
            votes.append(parsed)
            
            self._log("ensemble_vote", f"голос {i+1} результат", {
                "answer": answer, 
                "parsed": parsed,
                "temperature": temp,
                "llm_cot_generation": llm_cot_generation,
                "llm_few_shot_generation": llm_few_shot_generation
            }, "DEEP_DEBUG")
        
        from collections import Counter
        vote_counts = Counter(votes)
        most_common = vote_counts.most_common(1)[0]
        
        self._log("ensemble_vote", "результат", {
            "votes": votes,
            "distribution": dict(vote_counts),
            "final": most_common[0],
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")
        
        return most_common[0], dict(vote_counts)

    def ask_with_verification(
        self,
        question,
        encoded_options,
        category,
        temperature=0.1,
        use_llm_parsing=None,
        use_selfcheck=None,
        llm_cot_generation=None,  
        llm_few_shot_generation=None  
    ):
        if use_llm_parsing is None:
            use_llm_parsing = self.USE_LLM_PARSING
        if use_selfcheck is None:
            use_selfcheck = self.USE_SELFCHECK
        if llm_cot_generation is None:
            llm_cot_generation = self.LLM_COT_GENERATION
        if llm_few_shot_generation is None:
            llm_few_shot_generation = self.LLM_FEW_SHOT_GENERATION
        
        self._log("ask_with_verification", "начало", {
            "category": category,
            "temperature": temperature,
            "use_llm_parsing": use_llm_parsing,
            "use_selfcheck": use_selfcheck,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")
        
        first_answer = self.generate_answer(
            question=question,
            encoded_options=encoded_options,
            category=category,
            temperature=temperature,
            tokens=400,
            use_llm_parsing=use_llm_parsing,
            use_selfcheck=use_selfcheck,
            llm_cot_generation=llm_cot_generation,
            llm_few_shot_generation=llm_few_shot_generation
        )
        
        self._log("ask_with_verification", "первый ответ", {
            "first_answer": first_answer,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEEP_DEBUG")
        
        res = self._generate_selfcheck_prompt(
            question=question,
            topic=category,
            encoded_options=encoded_options,
            predicted_answer=first_answer,
            drammatic=True,
            use_llm_parsing=use_llm_parsing,
            llm_cot_generation=llm_cot_generation,
            llm_few_shot_generation=llm_few_shot_generation
        )

        if res is None:
            self._log("ask_with_verification", "ошибка генерации selfcheck", None, "DEBUG")
            return 0, 0, 0
        
        system_prompt, user_prompt = res
        check_response = self.direct_prompt(
            user_prompt=user_prompt,
            system_prompt=system_prompt,
            tokens=300,
            temperature=0.1
        )
        
        verified_answer = self.parse_answer_index(check_response, use_llm_parsing=use_llm_parsing)
        
        weight = 1.5 if first_answer == verified_answer else 0.7
        
        self._log("ask_with_verification", "результат", {
            "first": first_answer,
            "verified": verified_answer,
            "weight": weight,
            "agreement": first_answer == verified_answer,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")
        
        return verified_answer, weight, first_answer

    def confidence_ensemble_vote(
        self,
        question,
        encoded_options,
        category,
        n_runs=3,
        temperatures=[0.1, 0.3, 0.5],
        use_llm_parsing=None,
        use_selfcheck=None,
        llm_cot_generation=None,  
        llm_few_shot_generation=None  
    ):
        if use_llm_parsing is None:
            use_llm_parsing = self.USE_LLM_PARSING
        if use_selfcheck is None:
            use_selfcheck = self.USE_SELFCHECK
        if llm_cot_generation is None:
            llm_cot_generation = self.LLM_COT_GENERATION
        if llm_few_shot_generation is None:
            llm_few_shot_generation = self.LLM_FEW_SHOT_GENERATION
        
        self._log("confidence_ensemble_vote", "начало", {
            "category": category,
            "n_runs": n_runs,
            "temperatures": temperatures[:n_runs],
            "use_llm_parsing": use_llm_parsing,
            "use_selfcheck": use_selfcheck,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")
        
        results = []
        
        for i, temp in enumerate(temperatures[:n_runs]):
            self._log("confidence_ensemble_vote", f"запуск {i+1}", {
                "temperature": temp,
                "llm_cot_generation": llm_cot_generation,
                "llm_few_shot_generation": llm_few_shot_generation
            }, "DEEP_DEBUG")
            
            answer, weight, first_answer = self.ask_with_verification(
                question=question,
                encoded_options=encoded_options,
                category=category,
                temperature=temp,
                use_llm_parsing=use_llm_parsing,
                use_selfcheck=use_selfcheck,
                llm_cot_generation=llm_cot_generation,
                llm_few_shot_generation=llm_few_shot_generation
            )
            
            results.append({
                'answer': answer,
                'weight': weight,
                'first_answer': first_answer,
                'agreement': first_answer == answer
            })
            
            self._log("confidence_ensemble_vote", f"запуск {i+1} результат", {
                "answer": answer,
                "weight": weight,
                "agreement": first_answer == answer,
                "llm_cot_generation": llm_cot_generation,
                "llm_few_shot_generation": llm_few_shot_generation
            }, "DEEP_DEBUG")
        
        weighted_counts = {}
        for res in results:
            ans = res['answer']
            weight = res['weight']
            if ans in weighted_counts:
                weighted_counts[ans] += weight
            else:
                weighted_counts[ans] = weight
        
        best_answer = max(weighted_counts.items(), key=lambda x: x[1])
        total_weight = sum(res['weight'] for res in results)
        final_confidence = best_answer[1] / total_weight if total_weight > 0 else 0
        
        agreement_rate = sum(1 for res in results if res['agreement']) / len(results) if len(results) > 0 else 0
        
        self._log("confidence_ensemble_vote", "финальный результат", {
            "best_answer": best_answer[0],
            "confidence": final_confidence,
            "agreement_rate": agreement_rate,
            "weighted_counts": weighted_counts,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")
        
        return best_answer[0], weighted_counts, final_confidence, agreement_rate

    def evaluate_dataframe(
        self,
        df,
        question_column="question",
        options_column="options",
        category_column="category",
        answer_column="answer",
        method:Literal[
            "ensemble_vote",
            "generate_answer",
            "confidence_ensemble_vote",
            "generate_answer_selfcheck"
        ]="ensemble_vote",
        method_kwargs=None,
        use_llm_parsing=None,
        use_selfcheck=None,
        llm_cot_generation=None,  
        llm_few_shot_generation=None  
    ):
        if method_kwargs is None:
            method_kwargs = {}
        
        if use_llm_parsing is None:
            use_llm_parsing = self.USE_LLM_PARSING
        if use_selfcheck is None:
            use_selfcheck = self.USE_SELFCHECK
        if llm_cot_generation is None:
            llm_cot_generation = self.LLM_COT_GENERATION
        if llm_few_shot_generation is None:
            llm_few_shot_generation = self.LLM_FEW_SHOT_GENERATION
        
        self._log("evaluate_dataframe", "начало", {
            "rows": len(df),
            "method": method,
            "columns": df.columns.tolist(),
            "use_llm_parsing": use_llm_parsing,
            "use_selfcheck": use_selfcheck,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")
        
        predictions = []
        processing_times = []
        category_results = {}
        
        pbar = tqdm(total=len(df), desc="Обработка вопросов")
        
        for idx, row in df.iterrows():
            try:
                start_time = time.time()
                
                question = str(row[question_column])
                options = row[options_column]
                category = str(row[category_column])
                
                expected_answer = None
                if answer_column in df.columns and pd.notna(row[answer_column]):
                    expected_answer = self.parse_answer_index(str(row[answer_column]))
                
                self._log("evaluate_dataframe", f"строка {idx}", {
                    "category": category,
                    "expected": expected_answer,
                    "question": question[:50],
                    "use_llm_parsing": use_llm_parsing,
                    "use_selfcheck": use_selfcheck,
                    "llm_cot_generation": llm_cot_generation,
                    "llm_few_shot_generation": llm_few_shot_generation
                }, "DEEP_DEBUG")
                
                method_kwargs_with_defaults = {
                    "use_llm_parsing": use_llm_parsing,
                    "use_selfcheck": use_selfcheck,
                    "llm_cot_generation": llm_cot_generation,  
                    "llm_few_shot_generation": llm_few_shot_generation,  
                    **method_kwargs
                }
                
                if method == "generate_answer":
                    predicted = self.generate_answer(
                        question=question,
                        encoded_options=options,
                        category=category,
                        **method_kwargs_with_defaults
                    )
                elif method == "generate_answer_selfcheck":
                    predicted = self.generate_answer_selfcheck(
                        question=question,
                        encoded_options=options,
                        category=category,
                        **method_kwargs_with_defaults
                    )
                elif method == "ensemble_vote":
                    predicted, vote_dist = self.ensemble_vote(
                        question=question,
                        encoded_options=options,
                        category=category,
                        **method_kwargs_with_defaults
                    )
                    
                    if self.DEEP_DEBUG:
                        print(f"ensemble_vote распределение: {vote_dist}")
                elif method == "confidence_ensemble_vote":
                    predicted, weighted_counts, confidence, agreement = self.confidence_ensemble_vote(
                        question=question,
                        encoded_options=options,
                        category=category,
                        **method_kwargs_with_defaults
                    )
                    
                    if self.DEEP_DEBUG:
                        print(f"confidence_ensemble_vote: confidence={confidence}, agreement={agreement}")
                else:
                    raise ValueError(f"Неизвестный метод: {method}")
                
                predictions.append(predicted)
                processing_time = time.time() - start_time
                processing_times.append(processing_time)
                
                if category not in category_results:
                    category_results[category] = {
                        'total': 0, 'correct': 0, 'predictions': [], 'truths': []
                    }
                
                category_results[category]['total'] += 1
                category_results[category]['predictions'].append(predicted)
                
                if expected_answer is not None:
                    category_results[category]['truths'].append(expected_answer)
                    predicted_parsed = self.parse_answer_index(str(predicted))
                    
                    if predicted_parsed == expected_answer:
                        category_results[category]['correct'] += 1
                        
                        if self.DEEP_DEBUG:
                            print(f"✓ Правильно: предсказано {predicted_parsed}, ожидалось {expected_answer}")
                    else:
                        if self.DEEP_DEBUG:
                            print(f"✗ Ошибка: предсказано {predicted_parsed}, ожидалось {expected_answer}")
                
                pbar.update(1)
                
                pbar.set_postfix({
                    'категория': category[:10],
                    'предсказано': self.parse_answer_index(str(predicted)),
                    'ответ': expected_answer if expected_answer is not None else '?'
                })
                
            except Exception as e:
                print(f"\nОшибка в строке {idx}: {e}")
                
                self._log("evaluate_dataframe", f"ошибка в строке {idx}", {
                    "error": str(e),
                    "llm_cot_generation": llm_cot_generation,
                    "llm_few_shot_generation": llm_few_shot_generation
                }, "DEBUG")
                
                predictions.append(0)
                processing_times.append(0)
                pbar.update(1)
        
        pbar.close()
        
        self._log("evaluate_dataframe", "завершено", {
            "total_rows": len(df),
            "predictions_made": len(predictions),
            "use_llm_parsing": use_llm_parsing,
            "use_selfcheck": use_selfcheck,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")
        
        results_df = df.copy()
        results_df['predicted'] = predictions
        results_df['processing_time'] = processing_times
        
        if answer_column in df.columns:
            try:
                results_df['predicted_parsed'] = results_df['predicted'].apply(
                    lambda x: self.parse_answer_index(str(x)) if pd.notna(x) else 0
                )
                results_df['answer_parsed'] = results_df[answer_column].apply(
                    lambda x: self.parse_answer_index(str(x)) if pd.notna(x) else 0
                )
                results_df['is_correct'] = results_df['predicted_parsed'] == results_df['answer_parsed']
                
                if self.DEBUG:
                    correct = results_df['is_correct'].sum()
                    total = len(results_df)
                    accuracy = correct / total if total > 0 else 0
                    print(f"📊 Итог: {correct}/{total} правильных ({accuracy:.2%})")
                    print(f"⚙️ Настройки: use_llm_parsing={use_llm_parsing}, use_selfcheck={use_selfcheck}")
                    print(f"⚙️ CoT генерация: {'ВКЛ' if llm_cot_generation else 'ВЫКЛ'}")
                    print(f"⚙️ Few-shot генерация: {'ВКЛ' if llm_few_shot_generation else 'ВЫКЛ'}")
                    
            except Exception as e:
                print(f"Ошибка при создании метрик: {e}")
                results_df['is_correct'] = False
        
        metrics = self._calculate_metrics(
            results_df,
            answer_column,
            category_results,
            processing_times,
            method
        )
        
        metrics['use_llm_parsing'] = use_llm_parsing
        metrics['use_selfcheck'] = use_selfcheck
        metrics['llm_cot_generation'] = llm_cot_generation  
        metrics['llm_few_shot_generation'] = llm_few_shot_generation  
        
        if self.DEBUG:
            accuracy = metrics.get('accuracy', 0)
            print(f"📈 Точность: {accuracy:.2%}")
            print(f"⚙️ Параметры: LLM парсинг={use_llm_parsing}, Self-check={use_selfcheck}")
            print(f"⚙️ CoT генерация: {'ВКЛ' if llm_cot_generation else 'ВЫКЛ'}")
            print(f"⚙️ Few-shot генерация: {'ВКЛ' if llm_few_shot_generation else 'ВЫКЛ'}")
        
        return results_df, metrics
    
    

    def generate_prompt(
        self,
        question,
        topic,
        encoded_options,
        drammatic:bool = True,
        drammatic_prompt = "На тебя возложена огромная надежда и ответственность за мою жизнь и репутацию, не подведи нас",
        few_shot = True,
        llm_cot_generation=True,  
        llm_few_shot_generation=True  
    ):
        self._log("generate_prompt", "начало", {
            "topic": topic, 
            "few_shot": few_shot,
            "encoded_options_raw": str(encoded_options)[:200],
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEEP_DEBUG")
        
        options = self._options_parser(options=encoded_options)
        
        self._log("generate_prompt", "распарсенные опции", {
            "count": len(options),
            "options_preview": options[:3] if options else []
        }, "DEBUG")
        
        if not options or (len(options) == 1 and options[0] == "Варианты не предоставлены"):
            self._log("generate_prompt", "ВНИМАНИЕ: нет вариантов ответа!", {
                "raw_input": str(encoded_options)[:500]
            }, "DEBUG")
            return None
        
        base_prompt = self.prompts.get(topic,
            "Ты - опытный специалист широкого профиля. Ответь на предоставленный тебе вопрос, выбрав индекс правильного ответа (начиная с нуля)"
            "Дай в ответе только индекс, не давай дополнительных комментариев"
            "Для решения задачи обдумай каждый вариант ответа, подумай, почему он может быть правильным или неправильным"
        )
        
        
        cot_section = ""
        if llm_cot_generation:
            cot_instruction = self.generate_cot_instruction(
                question=question,
                topic=topic,
                encoded_options=encoded_options,
                max_tokens=400
            )
            
            if cot_instruction:
                cot_section = f"""
                
ИНСТРУКЦИЯ ДЛЯ РЕШЕНИЯ (CHAIN OF THOUGHT):
Реши задачу, следуя этим шагам:
{cot_instruction}

ВАЖНО: После анализа напиши "ОКОНЧАТЕЛЬНЫЙ ОТВЕТ: [индекс]"
"""
            else:
                self._log("generate_prompt", "CoT не сгенерирована, используем fallback", None, "DEBUG")
                cot_section = """
                
ИНСТРУКЦИЯ ДЛЯ РЕШЕНИЯ:
1. Внимательно прочитай вопрос
2. Проанализируй каждый вариант ответа
3. Исключи неправильные варианты
4. Выбери правильный вариант
5. Напиши только индекс правильного ответа
"""
        
        
        few_shot_text = ""
        if few_shot:
            if llm_few_shot_generation:
                few_shot_text = self.generate_contextual_few_shot(
                    question=question,
                    topic=topic,
                    encoded_options=encoded_options,
                    num_examples=2
                )
            else:
                few_shot_text = self.few_shot_prompts.get(topic,
                    """Вот примеры ответов:
Пример 1:
Вопрос: Какая самая длинная река в мире?
Варианты ответа:
0. Амазонка
1. Нил
2. Янцзы
3. Миссисипи
Ответ: 1

Пример 2:
Вопрос: Кто написал "Войну и мир"?
Варианты ответа:
0. Достоевский
1. Толстой
2. Чехов
3. Тургенев
Ответ: 1""" if few_shot else ""
                )
        
        system_prompt = f"""
{base_prompt}

{f"Примеры решения подобных задач:\n{few_shot_text}\n" if few_shot_text else ""}
{cot_section if llm_cot_generation else ""}

ОБРАТИ ВНИМАНИЕ:
- В этом вопросе {len(options)} вариантов (индексы от 0 до {len(options)-1})
- В ответе дай ТОЛЬКО индекс правильного варианта
- Не пиши объяснений, только число

ФОРМАТ ОТВЕТА: [индекс] (например: 2)
"""
        
        options_text = "\n".join([f"{ind}. {opt}" for ind, opt in enumerate(options)])
        
        user_prompt = f"""
Вопрос: {question}

Варианты ответа:
{options_text}

{drammatic_prompt if drammatic else ""}

Ответ (только индекс):"""
        
        self._log("generate_prompt", "сгенерирован", {
            "options_count": len(options),
            "question_preview": question[:100],
            "first_option": options[0][:100] if options else "нет",
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation,
            "has_cot": bool(cot_section),
            "has_few_shot": bool(few_shot_text)
        }, "DEBUG")
        
        if self.DEEP_DEBUG:
            print("\n[DEEP_DEBUG] generate_prompt - финальный промпт:")
            print(f"System preview: {system_prompt[:300]}...")
            print(f"\nUser preview: {user_prompt[:300]}...")
        
        return system_prompt, user_prompt

    def _generate_selfcheck_prompt(
        self,
        question,
        topic,
        encoded_options,
        predicted_answer,
        drammatic:bool = True,
        drammatic_prompt = "На тебя возложена огромная надежда и ответственность за мою жизнь и репутацию, не подведи нас",
        few_shot = True,
        use_llm_parsing=True,
        llm_cot_generation=True,  
        llm_few_shot_generation=True  
    ):
        self._log("_generate_selfcheck_prompt", "начало", {
            "topic": topic,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEEP_DEBUG")
        
        options = self._options_parser(options=encoded_options)
        if not options:
            self._log("_generate_selfcheck_prompt", "ошибка парсинга опций", None, "DEBUG")
            return None
        
        predicted_idx = self.parse_answer_index(
            str(predicted_answer),
            use_llm_parsing=use_llm_parsing
        )
        
        option_text = "неизвестный вариант"
        if 0 <= predicted_idx < len(options):
            option_text = options[predicted_idx]
        
        self._log("_generate_selfcheck_prompt", "индекс предсказания", {
            "raw": str(predicted_answer)[:50],
            "parsed": predicted_idx,
            "option_text": option_text[:50],
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEEP_DEBUG")
        
        base_prompt = """Ты - эксперт по проверке ответов. Твоя задача - проверить ответ другого эксперта.
Если он правильный - верни тот же индекс. Если неправильный - найди и верни правильный индекс."""
        
        if llm_cot_generation:
            base_prompt += "\n\nИСПОЛЬЗУЙ ЦЕПОЧКУ РАССУЖДЕНИЙ:\n"
            base_prompt += "1. Проанализируй оригинальный вопрос\n"
            base_prompt += "2. Проверь ответ коллеги на соответствие вопросу\n"
            base_prompt += "3. Если сомневаешься, найди правильный ответ самостоятельно\n"
            base_prompt += "4. Дай окончательный ответ в формате 'ОТВЕТ: [индекс]'"
        
        if few_shot:
            base_prompt += """
Пример 1:
Вопрос: Какая столица Франции?
Варианты:
0. Лондон
1. Берлин
2. Париж
3. Мадрид
Ответ коллеги: 2
Твой ответ: 2

Пример 2:
Вопрос: 2 + 2 = ?
Варианты:
0. 3
1. 4
2. 5
3. 6
Ответ коллеги: 0
Твой ответ: 1"""
        
        system_prompt = f"{base_prompt}"
        
        options_text = "\n".join([f"{i}. {opt}" for i, opt in enumerate(options)])
        
        user_prompt = f"""
Вопрос: {question}
Варианты ответов:
{options_text}
Ответ коллеги: {predicted_idx} ({option_text})
{drammatic_prompt if drammatic else ""}
Твой ответ (только индекс):
ОБРАТИ ВНИМАНИЕ - В ОТВЕТЕ ТЕБЕ НУЖНО ДАТЬ ОДИН ИНДЕКС
НЕ ПИШИ ДОПОЛНИТЕЛЬНЫХ РАССУЖДЕНИЙ
"""
        
        self._log("_generate_selfcheck_prompt", "сгенерирован", {
            "options_count": len(options),
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")
        
        return system_prompt, user_prompt

    def _log(self, method, message, data=None, level="DEBUG"):
        if level == "DEBUG" and not self.DEBUG:
            return
        if level == "DEEP_DEBUG" and not self.DEEP_DEBUG:
            return
        
        log_entry = {
            "timestamp": time.time(),
            "method": method,
            "message": message,
            "data": data,
            "level": level
        }
        self.debug_logs.append(log_entry)
        
        if self.DEBUG or self.DEEP_DEBUG:
            print(f"[{level}] {method}: {message}")
            if data and self.DEEP_DEBUG:
                print(f"    Данные: {data}")
    
    def _log_response(self, stage, raw_response, parsed, expected=None, metadata=None):
        if not self.DEEP_DEBUG:
            return
        
        print(f"\n{'='*80}")
        print(f"[DEEP_DEBUG] {stage}")
        print(f"Сырой ответ ({len(raw_response)} chars):")
        print(f"{raw_response[:500]}...")
        print(f"Распарсено: {parsed}")
        if expected is not None:
            print(f"Ожидалось: {expected}")
            print(f"Совпадение: {parsed == expected}")
        if metadata:
            print(f"Метаданные: {metadata}")
        print(f"{'='*80}\n")
        
        log_entry = {
            "timestamp": time.time(),
            "stage": stage,
            "raw_response": raw_response[:1000],
            "parsed": parsed,
            "expected": expected,
            "metadata": metadata
        }
        self.debug_logs.append(log_entry)

    def _options_parser(self, options):
        self._log("_options_parser", "начало", {
            "raw_input": str(options)[:200],
            "type": type(options)
        }, "DEEP_DEBUG")
        
        
        if isinstance(options, list):
            self._log("_options_parser", "уже список", {"len": len(options), "first_3": options[:3]}, "DEEP_DEBUG")
            return options
        
        original_input = str(options)
        
        
        if isinstance(options, str):
            
            text = original_input.strip()
            
            
            if text.startswith('[') and text.endswith(']'):
                try:
                    
                    json_text = text.replace("'", '"')
                    parsed = json.loads(json_text)
                    if isinstance(parsed, list):
                        self._log("_options_parser", "JSON парсинг успешен", {"len": len(parsed)}, "DEBUG")
                        return parsed
                except json.JSONDecodeError as e:
                    self._log("_options_parser", "JSON ошибка", {"error": str(e)}, "DEBUG")
            
            
            if text.startswith('[') and text.endswith(']'):
                
                content = text[1:-1].strip()
                self._log("_options_parser", "формат с пробелами", {"content_preview": content[:100]}, "DEEP_DEBUG")
                
                items = []
                current_item = ""
                in_quotes = False
                quote_char = None
                
                i = 0
                while i < len(content):
                    char = content[i]
                    
                    if char in ['"', "'"]:
                        if not in_quotes:
                            
                            in_quotes = True
                            quote_char = char
                            current_item += char
                        elif char == quote_char:
                            
                            in_quotes = False
                            current_item += char
                            items.append(current_item)
                            current_item = ""
                            
                            
                            i += 1
                            while i < len(content) and content[i] in [' ', '\n', '\t']:
                                i += 1
                            continue
                        else:
                            current_item += char
                    elif char == ' ' and not in_quotes:
                        
                        if current_item:
                            items.append(current_item)
                            current_item = ""
                    else:
                        current_item += char
                    
                    i += 1
                
                
                if current_item:
                    items.append(current_item)
                
                
                cleaned_items = []
                for item in items:
                    item = item.strip()
                    if item:
                        if (item.startswith('"') and item.endswith('"')) or \
                        (item.startswith("'") and item.endswith("'")):
                            item = item[1:-1]
                        
                        item = item.replace('\\"', '"').replace("\\'", "'").replace('\\n', '\n')
                        cleaned_items.append(item)
                
                if cleaned_items:
                    self._log("_options_parser", "специальный формат распарсен", {
                        "count": len(cleaned_items),
                        "first_3": cleaned_items[:3]
                    }, "DEBUG")
                    return cleaned_items
        
        
        if isinstance(options, str) and ',' in options:
            try:
                
                parts = []
                current = ""
                in_quotes = False
                quote_char = None
                
                for char in options:
                    if char in ['"', "'"]:
                        if not in_quotes:
                            in_quotes = True
                            quote_char = char
                        elif char == quote_char:
                            in_quotes = False
                        current += char
                    elif char == ',' and not in_quotes:
                        parts.append(current.strip())
                        current = ""
                    else:
                        current += char
                
                if current:
                    parts.append(current.strip())
                
                
                cleaned_parts = []
                for part in parts:
                    part = part.strip()
                    if part:
                        if (part.startswith('"') and part.endswith('"')) or \
                        (part.startswith("'") and part.endswith("'")):
                            part = part[1:-1]
                        cleaned_parts.append(part)
                
                if cleaned_parts:
                    self._log("_options_parser", "разделили по запятым", {
                        "count": len(cleaned_parts),
                        "first_3": cleaned_parts[:3]
                    }, "DEBUG")
                    return cleaned_parts
            except Exception as e:
                self._log("_options_parser", "ошибка при разделении по запятым", {"error": str(e)}, "DEBUG")
        
        
        if isinstance(options, str) and len(options) > 10:
            self._log("_options_parser", "пробуем LLM парсинг", None, "DEBUG")
            llm_parsed = self._llm_parse_options(options)
            if llm_parsed:
                return llm_parsed
        
        
        self._log("_options_parser", "не удалось распарсить", {
            "original_length": len(original_input),
            "original_preview": original_input[:200]
        }, "DEBUG")
        
        return ["Варианты не предоставлены"]

    def _llm_parse_options(self, options_text):
        """Использование LLM для парсинга сложных форматов опций"""
        try:
            system_prompt = """Ты ассистент для парсинга данных. Извлеки список вариантов ответа из текста.
    Текст может быть в разных форматах: JSON, Python список, или строковое представление.
    Верни ТОЛЬКО валидный JSON список строк.

    Пример 1:
    Вход: "['Вариант A' 'Вариант B' 'Вариант C']"
    Выход: ["Вариант A", "Вариант B", "Вариант C"]

    Пример 2:
    Вход: "['Южная Америка' 'Южная Азия' 'Северная Африка']"
    Выход: ["Южная Америка", "Южная Азия", "Северная Африка"]

    Пример 3:
    Вход: "['Верно, Неверно' 'Не указано, Не указано']"
    Выход: ["Верно, Неверно", "Не указано, Не указано"]

    ПРАВИЛА:
    1. Всегда возвращай валидный JSON
    2. Только список строк
    3. Сохраняй оригинальный текст вариантов
    4. Если не можешь распарсить - верни пустой список []"""

            user_prompt = f"""Извлеки список вариантов из текста:

    Текст: {options_text}

    JSON список:"""
            
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
            
            text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            inputs = self.tokenizer(text, return_tensors="pt").to(self.device)
            
            with torch.no_grad():
                generated_ids = self.model.generate(
                    inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=500,
                    temperature=0.1,
                    do_sample=False,
                    pad_token_id=self.tokenizer.pad_token_id
                )
            
            llm_response = self.tokenizer.decode(
                generated_ids[0][inputs.input_ids.shape[1]:], 
                skip_special_tokens=True
            )
            
            self._log("_llm_parse_options", "LLM ответ", {"response": llm_response[:200]}, "DEBUG")
            
            
            try:
                parsed = json.loads(llm_response)
                if isinstance(parsed, list):
                    self._log("_llm_parse_options", "успешно распарсено", {"count": len(parsed)}, "DEBUG")
                    return parsed
            except json.JSONDecodeError:
                
                import re
                json_match = re.search(r'\[.*\]', llm_response, re.DOTALL)
                if json_match:
                    try:
                        parsed = json.loads(json_match.group())
                        if isinstance(parsed, list):
                            self._log("_llm_parse_options", "нашли JSON в ответе", {"count": len(parsed)}, "DEBUG")
                            return parsed
                    except:
                        pass
            
            return []
            
        except Exception as e:
            self._log("_llm_parse_options", "ошибка", {"error": str(e)}, "DEBUG")
            return []

    def generate_cot_instruction(
        self,
        question: str,
        topic: str,
        encoded_options,
        max_tokens: int = 1000
    ) -> str:
        """
        Генерирует только CoT инструкцию для конкретного вопроса.
        
        Returns:
            str: Сгенерированная CoT инструкция или пустая строка при ошибке
        """
        self._log("generate_cot_instruction", "начало", {
            "topic": topic,
            "question_len": len(question),
            "max_tokens": max_tokens
        }, "DEBUG")
        
        options = self._options_parser(encoded_options)
        if not options or (len(options) == 1 and options[0] == "Варианты не предоставлены"):
            self._log("generate_cot_instruction", "нет опций", None, "DEBUG")
            return ""
        
        options_text = "\n".join([f"{i}. {opt}" for i, opt in enumerate(options)])
        
        system_prompt = """Ты эксперт по методике Chain of Thought (CoT). 
    Создай структурированную пошаговую инструкцию для решения ЗАДАЧИ.

    ТРЕБОВАНИЯ:
    1. Инструкция должна быть КОНКРЕТНОЙ для данной задачи
    2. Используй нумерованные шаги (1., 2., 3., ...)
    3. Включи логические проверки и анализ вариантов
    4. Учитывай специфику категории задачи
    5. Заверши четким указанием формата ответа

    ФОРМАТ ВЫВОДА:
    Начни сразу с пошаговой инструкции, без вступлений."""

        user_prompt = f"""Создай Chain of Thought инструкцию для решения:

    КАТЕГОРИЯ: {topic}

    ВОПРОС:
    {question}

    ВАРИАНТЫ ОТВЕТОВ (всего {len(options)}):
    {options_text}

    Инструкция должна помочь систематически решить эту задачу. 
    Индексы вариантов: от 0 до {len(options)-1}.

    CoT инструкция:"""
        
        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
            
            text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            inputs = self.tokenizer(text, return_tensors="pt").to(self.device)
            
            with torch.no_grad():
                generated_ids = self.model.generate(
                    inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=max_tokens,
                    temperature=0.3,
                    do_sample=False,
                    pad_token_id=self.tokenizer.pad_token_id,
                )
            
            cot_instruction = self.tokenizer.decode(
                generated_ids[0][inputs.input_ids.shape[1]:], 
                skip_special_tokens=True
            ).strip()
            
            self._log("generate_cot_instruction", "CoT сгенерирована", {
                "length": len(cot_instruction),
                "preview": cot_instruction[:200]
            }, "DEBUG")
            
            if self.DEEP_DEBUG:
                print(f"\n[DEEP_DEBUG] Сгенерированная CoT инструкция:")
                print(f"{cot_instruction}")
                print("-" * 80)
            
            return cot_instruction
            
        except Exception as e:
            self._log("generate_cot_instruction", "ошибка генерации", {"error": str(e)}, "DEBUG")
            return ""

    def generate_contextual_few_shot(
        self,
        question: str,
        topic: str,
        encoded_options,
        num_examples: int = 2,
        max_tokens_per_example: int = 1000
    ) -> str:
        """
        Генерирует few-shot примеры, тематически смежные с заданным вопросом.
        
        Args:
            question: Оригинальный вопрос (для определения тематики и сложности)
            topic: Категория вопроса
            encoded_options: Варианты ответов оригинального вопроса
            num_examples: Количество примеров для генерации
            max_tokens_per_example: Максимальное количество токенов на пример
        
        Returns:
            str: Сгенерированные few-shot примеры в формате строки
        """
        self._log("generate_contextual_few_shot", "начало", {
            "topic": topic,
            "num_examples": num_examples,
            "question_preview": question[:200]
        }, "DEBUG")

        options = self._options_parser(encoded_options)
        num_options = len(options) if options and options[0] != "Варианты не предоставлены" else 4
        
        question_complexity = "средней сложности" if len(question) > 100 else "базовой сложности"
        options_complexity = f"{num_options} вариантов" if num_options > 4 else "стандартное количество вариантов"

        system_prompt = f"""Ты эксперт по созданию few-shot инструкций для LLM по предмету "{topic}"
    Твоя задача - создать тематически смежные учебные примеры.

    ТРЕБОВАНИЯ К ПРИМЕРАМ:
    1. ТЕМАТИЧЕСКАЯ СМЕЖНОСТЬ: Примеры должны быть из той же узкой тематики, что и оригинальный вопрос
    2. СЛОЖНОСТЬ: Примеры должны быть примерно той же сложности
    3. ФОРМАТ: Каждый пример должен содержать:
    - Реалистичный вопрос по теме
    - Варианты ответов (от {max(3, num_options-2)} до {min(10, num_options+2)} вариантов)
    - Правильный ответ с индексом (ровно один)
    4. СТРУКТУРА: Один пример = один блок с четкой структурой

    ФОРМАТ ВЫВОДА для каждого примера:
    Пример [N]:
    Вопрос: [текст вопроса]
    Варианты ответа:
    0. [вариант 0]
    1. [вариант 1]
    ...
    [N]. [вариант N]
    Ответ: [индекс]

    Не пиши ничего кроме примеров."""

        question_analysis = f"""
    ОРИГИНАЛЬНЫЙ ВОПРОС (для анализа тематики и сложности):
    {question}

    ОПИСАНИЕ ОРИГИНАЛА:
    - Категория: {topic}
    - Сложность: {question_complexity}
    - Количество вариантов: {options_complexity}
    - Примерная тематика: {self._extract_topic_keywords(question, topic)}
    """

        user_prompt = f"""{question_analysis}

    Создай {num_examples} тематически смежных примера для few-shot обучения.

    УЧТИ:
    1. Тематика должна быть максимально близкой к оригинальному вопросу
    2. Сложность примеров должна быть сопоставимой
    3. Вариантов ответа: примерно {num_options} (от {max(3, num_options-2)} до {min(10, num_options+2)})
    4. Вопросы должны быть реалистичными и соответствовать предметной области
    5. Правильные ответы должны быть однозначными и проверяемыми

    Начни генерировать примеры:"""

        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
            
            text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            inputs = self.tokenizer(text, return_tensors="pt").to(self.device)
            
            with torch.no_grad():
                generated_ids = self.model.generate(
                    inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=max_tokens_per_example * num_examples,
                    temperature=0.4, 
                    do_sample=True,
                    top_p=0.9,
                    pad_token_id=self.tokenizer.pad_token_id,
                )
            
            few_shot_examples = self.tokenizer.decode(
                generated_ids[0][inputs.input_ids.shape[1]:], 
                skip_special_tokens=True
            ).strip()

            formatted_examples = self._validate_and_format_few_shot(few_shot_examples, topic, num_options)
            
            self._log("generate_contextual_few_shot", "примеры сгенерированы", {
                "original_topic": topic,
                "generated_length": len(formatted_examples),
                "num_examples_found": formatted_examples.count("Пример"),
                "preview": formatted_examples[:300]
            }, "DEBUG")
            
            if self.DEEP_DEBUG:
                print(f"\n[DEEP_DEBUG] Сгенерированные few-shot примеры для '{topic}':")
                print("-" * 80)
                print(formatted_examples)
                print("-" * 80)
            
            return formatted_examples
            
        except Exception as e:
            self._log("generate_contextual_few_shot", "ошибка генерации", {"error": str(e)}, "DEBUG")
            return ""

    def _extract_topic_keywords(self, question: str, topic: str) -> str:
        """Извлекает ключевые слова из вопроса для определения узкой тематики"""
        keywords = []
        
        if topic == 'math':
            math_terms = ['гомоморфизм', 'ядро', 'инъективный', 'кольцо', 'идеал', 'уравнение', 
                        'производная', 'интеграл', 'матрица', 'вектор', 'теорема', 'доказательство']
            keywords = [term for term in math_terms if term in question.lower()]
        
        elif topic == 'physics':
            physics_terms = ['скорость', 'ускорение', 'сила', 'энергия', 'заряд', 'волна',
                            'температура', 'давление', 'оптика', 'механика', 'электричество']
            keywords = [term for term in physics_terms if term in question.lower()]
        
        elif topic == 'history':
            history_terms = ['год', 'век', 'война', 'революция', 'договор', 'император',
                            'сражение', 'реформа', 'хронология', 'период', 'событие']
            keywords = [term for term in history_terms if term in question.lower()]
        
        if not keywords:
            words = question.lower().split()
            stop_words = {'этот', 'вопрос', 'относится', 'следующей', 'информации', 'какой', 'что', 'как'}
            keywords = [w for w in words if len(w) > 4 and w not in stop_words][:5]
        
        return ", ".join(keywords[:3]) if keywords else "общая тематика"

    def _validate_and_format_few_shot(self, examples_text: str, topic: str, expected_options: int) -> str:
        """Проверяет и форматирует сгенерированные few-shot примеры"""
        lines = examples_text.strip().split('\n')
        formatted = []
        current_example = []
        
        for line in lines:
            line = line.strip()
            if not line:
                continue

            if line.startswith(('Пример', 'пример', 'EXAMPLE', 'example')):
                if current_example:
                    formatted_example = self._format_single_example(current_example, topic, expected_options)
                    if formatted_example:
                        formatted.append(formatted_example)
                    current_example = []
                current_example.append(line)
            elif current_example:
                current_example.append(line)
        
        if current_example:
            formatted_example = self._format_single_example(current_example, topic, expected_options)
            if formatted_example:
                formatted.append(formatted_example)
        
        if not formatted:
            fallback = self._create_fallback_examples(topic, expected_options)
            return fallback
        
        return "\n\n".join(formatted)

    def _format_single_example(self, example_lines: list, topic: str, expected_options: int) -> str:
        """Форматирует один few-shot пример"""
        example_text = "\n".join(example_lines)
        
        has_question = any('вопрос:' in line.lower() for line in example_lines)
        has_options = any('вариант' in line.lower() for line in example_lines)
        has_answer = any('ответ:' in line.lower() for line in example_lines)
        
        if not (has_question and has_options and has_answer):
            return ""

        option_lines = [line for line in example_lines if re.match(r'^\d+\.', line.strip())]
        num_options = len(option_lines)

        if abs(num_options - expected_options) > 3 and expected_options > 3:
            return ""
        return example_text

    def _create_fallback_examples(self, topic: str, num_options: int) -> str:
        """Создает fallback примеры если генерация не удалась"""
        self._log("_create_fallback_examples", "создаем fallback", {
            "topic": topic,
            "num_options": num_options
        }, "DEBUG")
        

        fallbacks = self.few_shot_prompts
        if topic in fallbacks.keys():
            return fallbacks[topic]

        return f"""Пример 1:
    Вопрос: Пример вопроса по теме {topic}
    Варианты ответа:
    0. Вариант A
    1. Вариант B
    2. Вариант C
    {'' if num_options <= 3 else '3. Вариант D\n' + '\n'.join([f'{i}. Вариант {chr(65+i)}' for i in range(4, min(8, num_options))])}
    Ответ: 0"""

    def llm_parse_answer(self, raw_response: str) -> int:
        self._log("llm_parse_answer", "начало", {"raw_len": len(raw_response)}, "DEEP_DEBUG")
        
        system_prompt = """Ты - ассистент по извлечению чисел. Извлеки ЧИСЛО из текста.
Пример 1:
Текст: "Ответ: 2. Этот вариант правильный потому что..."
Извлеченное число: 2

Пример 2:
Текст: "Я считаю, что правильный вариант третий"
Извлеченное число: 2

Пример 3:
Текст: "Вариант А кажется верным"
Извлеченное число: 0

Пример 4:
Текст: "Номер правильного ответа: 5"
Извлеченное число: 4

ПРАВИЛА:
1. Извлекай ТОЛЬКО число (0, 1, 2, 3, ...)
2. Если в тексте несколько чисел - берем первое
3. Если указана буква (A=0, B=1, C=2, D=3)
4. Если не можешь извлечь - возвращай 0
5. Только число, без текста"""
        
        user_prompt = f"""Извлеки число из текста:

Текст: {raw_response}

Извлеченное число:"""
        
        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
            
            text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            inputs = self.tokenizer(text, return_tensors="pt").to(self.device)
            
            with torch.no_grad():
                generated_ids = self.model.generate(
                    inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=300,
                    temperature=0.1,
                    do_sample=False,
                    pad_token_id=self.tokenizer.pad_token_id
                )
            
            llm_parsed = self.tokenizer.decode(
                generated_ids[0][inputs.input_ids.shape[1]:], 
                skip_special_tokens=True
            )
            
            self._log("llm_parse_answer", "LLM ответ", {"llm_parsed": llm_parsed}, "DEEP_DEBUG")
            
            regex_parsed = self._regex_parse_answer(llm_parsed)
            
            self._log("llm_parse_answer", "финальный результат", {
                "raw_response_len": len(raw_response),
                "llm_parsed": llm_parsed,
                "regex_parsed": regex_parsed
            }, "DEBUG")
            
            return regex_parsed
            
        except Exception as e:
            self._log("llm_parse_answer", "ошибка", {"error": str(e)}, "DEBUG")
            return 0
    
    def _regex_parse_answer(self, text: str) -> int:
        import re
        
        text = str(text).strip()
        
        if not text:
            return 0
        
        try:
            num = int(text)
            return num
        except:
            pass
        
        patterns = [
            r'^\s*(\d+)\s*$',
            r'ответ[:\s]*(\d+)',
            r'число[:\s]*(\d+)',
            r'номер[:\s]*(\d+)',
            r'\b(\d+)\b',
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                try:
                    num = int(match.group(1))
                    return num
                except:
                    continue
        
        letter_match = re.search(r'\b([a-d])\b', text, re.IGNORECASE)
        if letter_match:
            letter = letter_match.group(1).upper()
            return ord(letter) - ord('A')
        
        return 0
    
    def parse_answer_index(self, answer_text: str, use_llm_parsing=True) -> int:
        self._log("parse_answer_index", "начало", {"raw": str(answer_text)[:100], "use_llm_parsing": use_llm_parsing}, "DEEP_DEBUG")
        
        if not answer_text:
            self._log("parse_answer_index", "пустой ответ", None, "DEBUG")
            return 0
        
        answer_text = str(answer_text).strip()
        
        try:
            num = int(answer_text)
            self._log("parse_answer_index", "прямое число", {"num": num}, "DEBUG")
            return num
        except:
            pass
        
        if use_llm_parsing and len(answer_text) > 10:
            llm_result = self.llm_parse_answer(answer_text)
            self._log("parse_answer_index", "LLM парсинг", {"llm_result": llm_result}, "DEBUG")
            
            if llm_result != 0:
                return llm_result
        
        regex_result = self._regex_parse_answer(answer_text)
        self._log("parse_answer_index", "регулярки", {"regex_result": regex_result}, "DEBUG")
        
        return regex_result

    def direct_prompt(
        self, 
        user_prompt:str,
        system_prompt:str,
        tokens:int = 1000,
        temperature:float = 0.1,
        few_shot = True
    ):
        self._log("direct_prompt", "начало", {
            "user_len": len(user_prompt),
            "system_len": len(system_prompt),
            "temperature": temperature
        }, "DEEP_DEBUG")
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        
        text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        self._log("direct_prompt", "шаблон применен", {"input_length": len(text)}, "DEEP_DEBUG")
        
        inputs = self.tokenizer(text, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            generated_ids = self.model.generate(
                inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=tokens,
                temperature=temperature,
                do_sample=False,
                pad_token_id=self.tokenizer.pad_token_id
            )
        
        response = self.tokenizer.decode(
            generated_ids[0][inputs.input_ids.shape[1]:], 
            skip_special_tokens=True
        )
        
        self._log("direct_prompt", "получен ответ", {"response_length": len(response)}, "DEBUG")
        
        if self.DEEP_DEBUG:
            print(f"[DEEP_DEBUG] direct_prompt response ({len(response)} chars):")
            print(f"{response[:500]}...")
        
        return response

    def get_debug_logs(self):
        return self.debug_logs

    def clear_debug_logs(self):
        self.debug_logs = []

    def save_debug_logs(self, filename="llm_debug_logs.json"):
        import json
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(self.debug_logs, f, ensure_ascii=False, indent=2)
        print(f"Логи сохранены в {filename}")

    def print_debug_summary(self):
        if not self.DEBUG and not self.DEEP_DEBUG:
            print("Отладка отключена")
            return
        
        print(f"\n{'='*60}")
        print("СВОДКА ОТЛАДКИ LLM")
        print(f"{'='*60}")
        print(f"Всего логов: {len(self.debug_logs)}")
        
        if self.debug_logs:
            methods = {}
            levels = {}
            for log in self.debug_logs:
                method = log.get("method", "unknown")
                level = log.get("level", "unknown")
                methods[method] = methods.get(method, 0) + 1
                levels[level] = levels.get(level, 0) + 1
            
            print("\nВызовы методов:")
            for method, count in sorted(methods.items()):
                print(f"  {method}: {count}")
            
            print("\nУровни логирования:")
            for level, count in sorted(levels.items()):
                print(f"  {level}: {count}")
            
            print(f"\nDEBUG: {self.DEBUG}")
            print(f"DEEP_DEBUG: {self.DEEP_DEBUG}")


    def _calculate_metrics(
        self,
        results_df,
        answer_column,
        category_results,
        processing_times,
        method
    ):
        metrics = {
            'method': method,
            'total_questions': len(results_df),
            'avg_processing_time': np.mean(processing_times) if processing_times else 0,
            'total_processing_time': sum(processing_times)
        }

        if answer_column in results_df.columns and 'is_correct' in results_df.columns:
            correct = results_df['is_correct'].sum()
            accuracy = correct / len(results_df) if len(results_df) > 0 else 0

            metrics.update({
                'correct_answers': int(correct),
                'accuracy': accuracy,
                'accuracy_percent': f"{accuracy * 100:.2f}%",
                'error_rate': 1 - accuracy
            })

            category_metrics = {}
            for category, results in category_results.items():
                if results['truths']:
                    cat_total = len(results['truths'])
                    cat_correct = results['correct']
                    cat_accuracy = cat_correct / cat_total if cat_total > 0 else 0

                    category_metrics[category] = {
                        'total': cat_total,
                        'correct': cat_correct,
                        'accuracy': cat_accuracy,
                        'accuracy_percent': f"{cat_accuracy * 100:.2f}%"
                    }

            metrics['category_metrics'] = category_metrics

            confusion = np.zeros((4, 4), dtype=int)
            for idx, row in results_df.iterrows():
                if pd.notna(row[answer_column]) and pd.notna(row['predicted']):
                    try:
                        true_val = self.parse_answer_index(str(row[answer_column]))
                    except:
                        true_val = 0
                    try:
                        pred_val = self.parse_answer_index(str(row['predicted']))
                    except:
                        pred_val = 0
                    
                    if 0 <= true_val < 4 and 0 <= pred_val < 4:
                        confusion[true_val][pred_val] += 1

            metrics['confusion_matrix'] = confusion.tolist()

            per_class_accuracy = []
            for i in range(4):
                total_class = sum(confusion[i])
                if total_class > 0:
                    class_acc = confusion[i][i] / total_class
                    per_class_accuracy.append({
                        'class': i,
                        'correct': int(confusion[i][i]),
                        'total': int(total_class),
                        'accuracy': class_acc,
                        'accuracy_percent': f"{class_acc * 100:.2f}%"
                    })

            metrics['per_class_accuracy'] = per_class_accuracy

        return metrics

    def process_csv_files(
        self,
        questions_csv_path: str,
        answers_csv_path: Optional[str] = None,
        output_dir: str = "./results",
        output_filename: Optional[str] = None,
        method:Literal[
            "ensemble_vote",
            "generate_answer",
            "confidence_ensemble_vote",
            "generate_answer_selfcheck"
        ]="ensemble_vote",
        save_intermediate: bool = True,
        **method_kwargs
    ) -> pd.DataFrame:
        """
        Обрабатывает CSV файлы с вопросами и ответами, запускает модель и сохраняет результаты.
        
        Args:
            questions_csv_path: Путь к CSV с вопросами (колонки: question, options, category)
            answers_csv_path: Опциональный путь к CSV с ответами (колонка: answer)
            output_dir: Директория для сохранения результатов
            output_filename: Имя файла результатов (если None - генерируется автоматически)
            method: Метод предсказания ('generate_answer', 'ensemble_vote', etc.)
            save_intermediate: Сохранять ли промежуточные результаты при ошибках
            **method_kwargs: Дополнительные параметры для метода
            
        Returns:
            pd.DataFrame: DataFrame с результатами
            
        Структура CSV:
        - Вопросы: безымянная колонка с номером, затем question, options, category
        - Ответы: безымянная колонка с номером, затем answer
        """
        
        self._log("process_csv_files", "начало обработки", {
            "questions_file": questions_csv_path,
            "answers_file": answers_csv_path,
            "output_dir": output_dir,
            "method": method
        }, "DEBUG")
        

        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        print(f"📁 Директория результатов: {output_path.absolute()}")
        
        print("📥 Загрузка данных...")
        try:
            questions_df = pd.read_csv(questions_csv_path)
            unnamed_cols = [col for col in questions_df.columns if 'Unnamed' in str(col)]
            if unnamed_cols:
                questions_df = questions_df.rename(columns={unnamed_cols[0]: 'question_id'})
                print(f"  ✓ Безымянная колонка переименована в 'question_id'")
            
            required_cols = ['question', 'options', 'category']
            missing_cols = [col for col in required_cols if col not in questions_df.columns]
            
            if missing_cols:
                raise ValueError(f"Отсутствуют необходимые колонки: {missing_cols}")
            
            questions_df['question_id'] = questions_df['question_id'].astype(int)
            questions_df['question'] = questions_df['question'].astype(str)
            questions_df['options'] = questions_df['options'].astype(str)
            questions_df['category'] = questions_df['category'].astype(str)
            
            print(f"  ✓ Загружено {len(questions_df)} вопросов")
            print(f"  ✓ Категории: {questions_df['category'].unique()[:5]}")
            
        except Exception as e:
            self._log("process_csv_files", "ошибка загрузки вопросов", {"error": str(e)}, "DEBUG")
            raise ValueError(f"Ошибка загрузки файла с вопросами: {e}")
        answers_df = None
        if answers_csv_path:
            try:
                answers_df = pd.read_csv(answers_csv_path)
                unnamed_cols = [col for col in answers_df.columns if 'Unnamed' in str(col)]
                if unnamed_cols:
                    answers_df = answers_df.rename(columns={unnamed_cols[0]: 'question_id'})
                
                if 'answer' not in answers_df.columns:
                    other_cols = [col for col in answers_df.columns if col != 'question_id']
                    if len(other_cols) == 1:
                        answers_df = answers_df.rename(columns={other_cols[0]: 'answer'})
                    else:
                        raise ValueError("Не могу найти колонку 'answer' в файле с ответами")
                answers_df['question_id'] = answers_df['question_id'].astype(int)
                answers_df['answer'] = answers_df['answer'].astype(str)
                
                print(f"  ✓ Загружено {len(answers_df)} ответов")
                
                q_ids = set(questions_df['question_id'])
                a_ids = set(answers_df['question_id'])
                
                if q_ids != a_ids:
                    missing_in_answers = q_ids - a_ids
                    missing_in_questions = a_ids - q_ids
                    
                    if missing_in_answers:
                        print(f"  ⚠️  ВНИМАНИЕ: Ответы отсутствуют для вопросов: {sorted(missing_in_answers)[:10]}")
                    if missing_in_questions:
                        print(f"  ⚠️  ВНИМАНИЕ: Лишние ответы для вопросов: {sorted(missing_in_questions)[:10]}")
                
            except Exception as e:
                self._log("process_csv_files", "ошибка загрузки ответов", {"error": str(e)}, "DEBUG")
                print(f"  ⚠️  Предупреждение: не удалось загрузить ответы: {e}")
                answers_df = None
        
        if answers_df is not None:
            merged_df = pd.merge(
                questions_df,
                answers_df[['question_id', 'answer']],
                on='question_id',
                how='left'
            )
            merged_df['has_answer'] = merged_df['answer'].notna()
            print(f"  ✓ Объединено: {merged_df['has_answer'].sum()} вопросов с ответами, "
                  f"{len(merged_df) - merged_df['has_answer'].sum()} без ответов")
        else:
            merged_df = questions_df.copy()
            merged_df['answer'] = None
            merged_df['has_answer'] = False
            print("  ✓ Ответы не предоставлены, будут сохранены только предсказания")
        
        print(f"\n🤖 Запуск модели (метод: {method})...")
        eval_kwargs = {
            "question_column": "question",
            "options_column": "options", 
            "category_column": "category",
            "answer_column": "answer" if 'answer' in merged_df.columns else None,
            "method": method,
            "method_kwargs": method_kwargs
        }

        eval_kwargs = {k: v for k, v in eval_kwargs.items() if v is not None}
        
        if save_intermediate:
            intermediate_path = output_path / "intermediate_data.csv"
            merged_df.to_csv(intermediate_path, index=False)
            print(f"  💾 Промежуточные данные сохранены: {intermediate_path}")
        
        try:
            results_df, metrics = self.evaluate_dataframe(
                merged_df,
                **eval_kwargs
            )
            
            print("\n✅ Обработка завершена!")
            print(f"📊 Обработано вопросов: {len(results_df)}")
            if 'is_correct' in results_df.columns:
                correct = results_df['is_correct'].sum()
                accuracy = correct / len(results_df) if len(results_df) > 0 else 0
                print(f"🎯 Точность: {correct}/{len(results_df)} ({accuracy:.2%})")
            
        except Exception as e:
            self._log("process_csv_files", "ошибка при оценке", {"error": str(e)}, "DEBUG")
            if save_intermediate and intermediate_path.exists():
                print("🔄 Загружаю промежуточные данные после ошибки...")
                results_df = pd.read_csv(intermediate_path)
                results_df['predicted'] = 0
                results_df['is_correct'] = False
                metrics = {'error': str(e)}
            else:
                raise RuntimeError(f"Ошибка при обработке данных: {e}")
        print("\n💾 Сохранение результатов...")
        if output_filename is None:
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            model_name_safe = self.model_name.replace("/", "_")
            output_filename = f"results_{model_name_safe}_{timestamp}.csv"
        
        output_filepath = output_path / output_filename
        
        final_columns = [
            'question_id', 'question', 'category', 
            'predicted', 'processing_time'
        ]
        
        if 'answer' in results_df.columns:
            final_columns.append('answer')
        if 'is_correct' in results_df.columns:
            final_columns.append('is_correct')
        if 'options' in results_df.columns:
            results_df['options_preview'] = results_df['options'].apply(
                lambda x: str(x)[:200] + "..." if len(str(x)) > 200 else str(x)
            )
            final_columns.append('options_preview')
        final_df = results_df[final_columns].copy()
        final_df.to_csv(output_filepath, index=False, encoding='utf-8-sig')
        
        print(f"  ✅ Результаты сохранены: {output_filepath}")
        print(f"  📄 Размер файла: {os.path.getsize(output_filepath) / 1024:.1f} KB")
        if metrics:
            stats_filename = output_filepath.with_suffix('.json')
            metadata = {
                "generated_at": time.strftime("%Y-%m-%d %H:%M:%S"),
                "questions_file": questions_csv_path,
                "answers_file": answers_csv_path,
                "model": self.model_name,
                "method": method,
                "method_kwargs": method_kwargs,
                "output_file": str(output_filepath),
                "metrics": metrics
            }
            import json
            with open(stats_filename, 'w', encoding='utf-8') as f:
                json.dump(metadata, f, ensure_ascii=False, indent=2)
            
            print(f"  📊 Метаданные сохранены: {stats_filename}")

        print(f"\n{'='*60}")
        print("СВОДКА ОБРАБОТКИ")
        print(f"{'='*60}")
        print(f"Обработано вопросов: {len(final_df)}")
        print(f"Категории: {', '.join(sorted(final_df['category'].unique()))}")
        
        if 'is_correct' in final_df.columns:
            accuracy = final_df['is_correct'].mean()
            print(f"Точность: {accuracy:.2%}")
            if len(final_df['category'].unique()) > 1:
                print("\n📈 Статистика по категориям:")
                for category in sorted(final_df['category'].unique()):
                    cat_df = final_df[final_df['category'] == category]
                    if len(cat_df) > 0 and 'is_correct' in cat_df.columns:
                        cat_acc = cat_df['is_correct'].mean()
                        print(f"  {category}: {cat_acc:.2%} ({len(cat_df)} вопросов)")
        
        avg_time = final_df['processing_time'].mean() if 'processing_time' in final_df.columns else 0
        print(f"Среднее время на вопрос: {avg_time:.2f} секунд")
        print(f"Файл результатов: {output_filepath}")
        print(f"{'='*60}")
        
        self._log("process_csv_files", "обработка завершена", {
            "output_file": str(output_filepath),
            "rows_processed": len(final_df),
            "has_answers": answers_csv_path is not None
        }, "DEBUG")
        
        return final_df


    def process_questions_only(
        self,
        questions_csv_path: str,
        output_dir: str = "./results",
        **kwargs
    ) -> pd.DataFrame:
        """Упрощенный вызов для обработки только вопросов без ответов"""
        return self.process_csv_files(
            questions_csv_path=questions_csv_path,
            answers_csv_path=None,
            output_dir=output_dir,
            **kwargs
        )



In [10]:
exp = LLM(
    model=model,
    tokenizer=tokenizer,
    deep_debug=True,
    use_llm_parsing=True,
    use_selfcheck=False,
    llm_few_shot_generation=False,
    llm_cot_generation=True,
)

🏋️‍♂️ Модель: Qwen/Qwen2.5-14B-Instruct
🖥 Устройство: cuda
⚙️  CoT генерация: ВКЛ
⚙️  Few-shot генерация: ВЫКЛ
✅ Используем переданные модель и токенайзер


In [11]:
exp.process_csv_files(
    questions_csv_path="/kaggle/input/llm-lab1/LR1_dev.csv",
    answers_csv_path="/kaggle/input/llm-lab1/LR1_dev_answers.csv",
    output_dir="/kaggle/working",
    method="generate_answer"
)

📁 Директория результатов: /kaggle/working
📥 Загрузка данных...
  ✓ Безымянная колонка переименована в 'question_id'
  ✓ Загружено 50 вопросов
  ✓ Категории: ['math' 'history' 'computer science' 'other' 'engineering']
  ✓ Загружено 50 ответов
  ✓ Объединено: 50 вопросов с ответами, 0 без ответов

🤖 Запуск модели (метод: generate_answer)...
  💾 Промежуточные данные сохранены: /kaggle/working/intermediate_data.csv


Обработка вопросов:   0%|          | 0/50 [00:00<?, ?it/s]

[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '7', 'use_llm_parsing': True}
[DEEP_DEBUG] evaluate_dataframe: строка 0
    Данные: {'category': 'math', 'expected': 7, 'question': 'Утверждение 1 | Гомоморфизм колец является инъекти', 'use_llm_parsing': True, 'use_selfcheck': False, 'llm_cot_generation': True, 'llm_few_shot_generation': False}
[DEEP_DEBUG] _options_parser: начало
    Данные: {'raw_input': "['Верно, Неверно' 'Не указано, Не указано' 'Неверно, Неверно'\n 'Не указано, Верно' 'Верно, Не указано' 'Не указано, Неверно'\n 'Верно, Верно' 'Неверно, Верно' 'Неверно, Не указано']", 'type': <class 'str'>}
[DEEP_DEBUG] _options_parser: формат с пробелами
    Данные: {'content_preview': "'Верно, Неверно' 'Не указано, Не указано' 'Неверно, Неверно'\n 'Не указано, Верно' 'Верно, Не указано"}
[DEEP_DEBUG] generate_prompt: начало
    Данные: {'topic': 'math', 'few_shot': True, 'encoded_options_raw': "['Верно, Неверно', 'Не указано, Не указано', 'Неверно, Неверно', 'Не указано

Обработка вопросов:   2%|▏         | 1/50 [00:35<28:56, 35.44s/it, категория=math, предсказано=0, ответ=7]

[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '0', 'use_llm_parsing': True}

[DEEP_DEBUG] MAIN_MODEL_RESPONSE
Сырой ответ (1 chars):
0...
Распарсено: 0
Метаданные: {'category': 'math', 'question': 'Утверждение 1 | Гомоморфизм колец является инъективным тогда и только тогда, когда его ядро состоит ', 'use_llm_parsing': True, 'use_selfcheck': False, 'llm_cot_generation': True, 'llm_few_shot_generation': False, 'temperature': 0.1, 'do_sample': False, 'response_length': 1}

[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '0', 'use_llm_parsing': True}
✗ Ошибка: предсказано 0, ожидалось 7
[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '0', 'use_llm_parsing': True}
[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '3', 'use_llm_parsing': True}
[DEEP_DEBUG] evaluate_dataframe: строка 1
    Данные: {'category': 'history', 'expected': 3, 'question': 'Этот вопрос относится к следующей информации.\nСлед', 'use_llm_parsing': True, 'use_selfcheck': Fa

...

Обработка вопросов:  98%|█████████▊| 49/50 [28:57<00:36, 36.17s/it, категория=biology, предсказано=3, ответ=5]

[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '3', 'use_llm_parsing': True}

[DEEP_DEBUG] MAIN_MODEL_RESPONSE
Сырой ответ (1 chars):
3...
Распарсено: 3
Метаданные: {'category': 'biology', 'question': 'Предположим, что в популяции гороха, находящейся в генетическом равновесии, частоты генов для полных', 'use_llm_parsing': True, 'use_selfcheck': False, 'llm_cot_generation': True, 'llm_few_shot_generation': False, 'temperature': 0.1, 'do_sample': True, 'response_length': 1}

[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '3', 'use_llm_parsing': True}
✗ Ошибка: предсказано 3, ожидалось 5
[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '3', 'use_llm_parsing': True}
[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '7', 'use_llm_parsing': True}
[DEEP_DEBUG] evaluate_dataframe: строка 49
    Данные: {'category': 'biology', 'expected': 7, 'question': 'Какое из следующих утверждений о теории когезии-те', 'use_llm_parsing': True, 'use_selfcheck': 

Обработка вопросов: 100%|██████████| 50/50 [29:33<00:00, 35.46s/it, категория=biology, предсказано=7, ответ=7]

[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '7', 'use_llm_parsing': True}

[DEEP_DEBUG] MAIN_MODEL_RESPONSE
Сырой ответ (1 chars):
7...
Распарсено: 7
Метаданные: {'category': 'biology', 'question': 'Какое из следующих утверждений о теории когезии-тензии для подъема воды в ксилеме является верным?', 'use_llm_parsing': True, 'use_selfcheck': False, 'llm_cot_generation': True, 'llm_few_shot_generation': False, 'temperature': 0.1, 'do_sample': True, 'response_length': 1}

[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '7', 'use_llm_parsing': True}
✓ Правильно: предсказано 7, ожидалось 7
[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '7', 'use_llm_parsing': True}
[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '0', 'use_llm_parsing': True}
[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '3', 'use_llm_parsing': True}
[DEEP_DEBUG] parse_answer_index: начало
    Данные: {'raw': '2', 'use_llm_parsing': True}
[DEEP_DEBUG] parse_an

,question_id,question,category,predicted,processing_time,answer,is_correct,options_preview
0,0,Утверждение 1 | Гомоморфизм колец является инъ...,math,0,35.435937,7,False,"['Верно, Неверно' 'Не указано, Не указано' 'Не..."
1,1,Этот вопрос относится к следующей информации.\...,history,3,40.118206,3,True,['Многие выступали за идею индивидуальных веро...
2,2,Этот вопрос основан на следующих объявлениях: ...,computer science,2,35.251773,2,True,['strC.compareTo(strB) > 0 && strB.compareTo(s...
3,3,Какое из следующих утверждений лучше всего опи...,other,8,35.154472,8,True,['Правительство может ограничить свободу верои...
4,4,Простые жители сельских ацтекских деревень Кап...,history,6,36.049314,2,False,['были обязаны платить высокие налоги товаром ...
5,5,Какой из перечисленных мировых регионов НЕ счи...,other,5,33.624158,0,False,['Южная Америка' 'Южная Азия' 'Северная Африка...
6,6,Найдите общее решение дифференциального уравне...,engineering,3,37.363921,0,False,['x=Y+A=c_1e^-t +c_2e^2t +a(cost - 2 sin t) +b...
7,7,Частица-источник совершает периодическое движе...,physics,9,35.965403,3,False,['13.3 см' '9 см' '80 см' '5.2 см' '10.6 см' '...
8,8,"Из лабораторных измерений мы знаем, что одна и...",physics,9,34.232015,9,True,['Яркость звезды колеблется.' 'Размер звезды у...
9,9,Иногда мальчики рождаются с неопущенными яичка...,biology,0,35.986963,0,True,['Человеческая сперма не может развиваться при...
